# Setup

In [ ]:
# installations

!pip install -q sentencepiece
!pip install -q transformers
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q textsum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
import os
import json
import zipfile
import evaluate
import torch
import functools as ft
import nltk
import networkx as nx

from collections import defaultdict
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from io import BytesIO, StringIO
from google.colab import drive, files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

from transformers import pipeline
from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration



In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# downloads

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

!ls
!pwd

--2023-04-08 18:26:17--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-08 18:26:17--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-08 18:26:18--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

# Functions

### Preprocessing functions

In [ ]:
# Preprocessing functions

def concatenate_script_element(file_data, script_element):
    '''concatenate only a specific script element such as dialog, text, speaker_heading, scene_heading'''
    combined_script_element = ''
    with open(file_data, 'r') as script_file:
        for line in script_file:
            if line.split(':')[0].strip() == script_element:
                combined_script_element += line.split(':')[1]
            else:
              continue

    # remove empty lines
    text = os.linesep.join([s for s in combined_script_element.splitlines() if s])

    return text

#####

def concatenate_dialog(file_data):
    '''concatenate only on dialog'''
    combined_dialog = ''
    with open(file_data, 'r') as script_file:
        for line in script_file:
            if line.split(':')[0].strip() == 'dialog':
                combined_dialog += line.split(':')[1]
            else:
              continue

    # remove empty lines
    text = os.linesep.join([s for s in combined_dialog.splitlines() if s])

    return text

#####

def concatenate_text(file_data):
    '''concatenate only on text'''
    combined_dialog = ''
    with open(file_data, 'r') as script_file:
        for line in script_file:
            if line.split(':')[0].strip() == 'text':
                combined_dialog += line.split(':')[1]
            else:
              continue

    # remove empty lines
    text = os.linesep.join([s for s in combined_dialog.splitlines() if s])

    return text

#####

def remove_stopwords(sen):
    ''' remove useless stopwords'''
    stop_words = stopwords.words('english')
    sen_new = " ".join([i for i in sen if i not in stop_words])

    return sen_new

#####

def clean_text(text):
    ''' remove punctuation, special characters, and numbers '''
    cleaned_text = re.sub(r'[^\w\s]|[\d_]+', '', text).strip()
    return cleaned_text

### Model functions

In [ ]:
def calculate_rouge_scores(prediction, reference):
    '''
    calculate ROUGE scores
    - prediction is a single string of long text
    - reference is the wikipedia plot_outline
    '''

    rouge = evaluate.load('rouge')
    predictions = [prediction]
    references = [reference]
    results = rouge.compute(predictions=predictions, references=references)

    return results

In [ ]:
def run_textrank_model(script, num_sentences_to_keep = int(50)):
    ''' 
    - PageRank to score sentences for extractive summary
    '''

    ### STEP 1 ###
    # TOKENIZE
    # tokenize words in each sentence
    sentences = sent_tokenize(script)
    print(len(sentences))

    # PREPROCESS
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).apply(clean_text)

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]

    # remove stopwords
    stop_words = stopwords.words('english') 
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

    # create sentence vectors with 100 elements each then average to get consolidated vector
    # https://www.kaggle.com/code/kunjan6902/text-summarization-textrank
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()]) / (len(i.split()) + 0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)

    ### STEP 2 ###
    # COSINE SIMILARITY MATRIX
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1, 100), sentence_vectors[j].reshape(1, 100))[0,0]

    ### STEP 3 ###
    # PAGERANK / TEXT RANK ALGORITHM
    # https://networkx.org/documentation/networkx-1.0/reference/generated/networkx.pagerank.html
    # https://arxiv.org/pdf/2108.02997.pdf
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph, alpha=0.85, tol=1e-02, max_iter=100)

    # ranked sentences
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    # get top X sentences based on provided funcion arguments
    top_sentences = ''
    for i in range(num_sentences_to_keep):
        top_sentences += (ranked_sentences[i][1])

    return top_sentences

In [ ]:
def run_pegasus_short_model(text_to_summarize):
    ''' 
    - max token length is 512 
    - Pegasus for Headline Generation
    '''

    # only use the first 1000 tokens
    text_to_summarize = text_to_summarize.strip()[:1000]

    pmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
    ptokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")    
    config = AutoConfig.from_pretrained("google/pegasus-xsum")
    inputs = ptokenizer(text_to_summarize, max_length=512, truncation=True, return_tensors="tf")

    # generate summary

    summary_ids = pmodel.generate(inputs["input_ids"],    
        max_length=256,
        min_length=94,
        no_repeat_ngram_size=3,
        num_beams=4)                                               

    prediction = ptokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    pprint(prediction, compact=True)

    return prediction

In [ ]:
def run_pegasus_long_model(text_to_summarize):
    ''' 
    - max token length is 1024 
    - Pegasus for Longer Generation
    '''

    # only use the first 1000 tokens
    text_to_summarize = text_to_summarize.strip()[:1000]

    cnnmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)
    cnntokenizer = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)
    config = AutoConfig.from_pretrained("google/pegasus-cnn_dailymail")
    cnninputs = cnntokenizer(text_to_summarize, max_length=1024, truncation=True, return_tensors="tf")

    # generate summary
    summary_ids = cnnmodel.generate(cnninputs["input_ids"], 
        max_length=256,
        min_length=94,
        no_repeat_ngram_size=3,
        num_beams=4)                            
                             
    prediction = cnntokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    pprint(prediction, compact=True)

    return prediction

In [ ]:
def run_pegasus_large_model(text_to_summarize):
    ''' 
    - max token length is 1024 
    - Pegasus Large
    '''

    # only use the first 1000 tokens
    text_to_summarize = text_to_summarize.strip()[:1000]

    cnnmodel = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-large", from_pt=True)
    cnntokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large", from_pt=True)
    config = AutoConfig.from_pretrained("google/pegasus-large")
    cnninputs = cnntokenizer(text_to_summarize, max_length=1024, truncation=True, return_tensors="tf")

    # generate summary
    summary_ids = cnnmodel.generate(cnninputs["input_ids"], 
        max_length=256,
        min_length=94,
        no_repeat_ngram_size=3,
        num_beams=4)
    
    prediction = cnntokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    pprint(prediction, compact=True)

    return prediction

In [ ]:
def run_longt5_model(text_to_summarize):
    ''' 
    - max token length is 16384 
    - LongT5 Booksum
    '''

    # only use the first 1000 tokens
    text_to_summarize = text_to_summarize.strip()[:1000]

    summarizer = pipeline(
        "summarization",
        "pszemraj/long-t5-tglobal-base-16384-book-summary",
        max_length=256,
        min_length=94,
        no_repeat_ngram_size=3,
        num_beams=4)

    prediction = summarizer(text_to_summarize)[0]["summary_text"]
    pprint(prediction, compact=True)

    return prediction

In [ ]:
def execute_two_stage_short_model(start, stop, rouge_dict, summary_dict, df):
    '''
    this function executes 
    1. textrank
    2. abstractive summarization (pegasus short models)
    3. rouge scoring
    '''

    for i in range(start, stop):

        ranked_sentences = run_textrank_model(df.iloc[i]['bert_dialog'])
        prediction = run_pegasus_short_model(ranked_sentences)
        reference = df.iloc[i]['plot_outline']
        rouge_scores = calculate_rouge_scores(prediction, reference)

        # add rouge scores and summary to a dictionary
        title = df.iloc[i]['title']
        rouge_dict[title] = rouge_scores
        summary_dict[title] = prediction

    return rouge_dict, summary_dict

In [ ]:
def execute_two_stage_long_model(start, stop, rouge_dict, summary_dict, df):
    '''
    this function executes 
    1. textrank
    2. abstractive summarization (pegasus long models)
    3. rouge scoring
    '''

    for i in range(start, stop):

        ranked_sentences = run_textrank_model(df.iloc[i]['bert_dialog'])
        prediction = run_pegasus_long_model(ranked_sentences)
        reference = df.iloc[i]['plot_outline']
        rouge_scores = calculate_rouge_scores(prediction, reference)

        # add rouge scores and summary to a dictionary
        title = df.iloc[i]['title']
        rouge_dict[title] = rouge_scores
        summary_dict[title] = prediction

    return rouge_dict, summary_dict

In [ ]:
def execute_two_stage_large_model(start, stop, rouge_dict, summary_dict, df):
    '''
    this function executes 
    1. textrank
    2. abstractive summarization (pegasus large models)
    3. rouge scoring
    '''

    for i in range(start, stop):

        ranked_sentences = run_textrank_model(df.iloc[i]['bert_dialog'])
        prediction = run_pegasus_large_model(ranked_sentences)
        reference = df.iloc[i]['plot_outline']
        rouge_scores = calculate_rouge_scores(prediction, reference)

        # add rouge scores and summary to a dictionary
        title = df.iloc[i]['title']
        rouge_dict[title] = rouge_scores
        summary_dict[title] = prediction

    return rouge_dict, summary_dict

In [ ]:
def execute_two_stage_t5_model(start, stop, rouge_dict, summary_dict, df):
    '''
    this function executes 
    1. textrank
    2. abstractive summarization (long t5 models)
    3. rouge scoring
    '''

    for i in range(start, stop):

        ranked_sentences = run_textrank_model(df.iloc[i]['bert_dialog'])
        prediction = run_longt5_model(ranked_sentences)
        reference = df.iloc[i]['plot_outline']
        rouge_scores = calculate_rouge_scores(prediction, reference)

        # add rouge scores and summary to a dictionary
        title = df.iloc[i]['title']
        rouge_dict[title] = rouge_scores
        summary_dict[title] = prediction

    return rouge_dict, summary_dict

In [ ]:
def execute_two_stage_text_short_model(start, stop, rouge_dict, summary_dict, df):
    '''
    this function executes 
    1. textrank
    2. abstractive summarization (pegasus short models)
    3. rouge scoring
    '''

    for i in range(start, stop):

        ranked_sentences = run_textrank_model(df.iloc[i]['bert_text'])
        prediction = run_pegasus_short_model(ranked_sentences)
        reference = df.iloc[i]['plot_outline']
        rouge_scores = calculate_rouge_scores(prediction, reference)

        # add rouge scores and summary to a dictionary
        title = df.iloc[i]['title']
        rouge_dict[title] = rouge_scores
        summary_dict[title] = prediction

    return rouge_dict, summary_dict

In [ ]:
def execute_two_stage_text_long_model(start, stop, rouge_dict, summary_dict, df):
    '''
    this function executes 
    1. textrank
    2. abstractive summarization (pegasus long models)
    3. rouge scoring
    '''

    for i in range(start, stop):

        ranked_sentences = run_textrank_model(df.iloc[i]['bert_text'])
        prediction = run_longt5_model(ranked_sentences)
        reference = df.iloc[i]['plot_outline']
        rouge_scores = calculate_rouge_scores(prediction, reference)

        # add rouge scores and summary to a dictionary
        title = df.iloc[i]['title']
        rouge_dict[title] = rouge_scores
        summary_dict[title] = prediction

    return rouge_dict, summary_dict

In [ ]:
def execute_two_stage_text_large_model(start, stop, rouge_dict, summary_dict, df):
    '''
    this function executes 
    1. textrank
    2. abstractive summarization (pegasus large models)
    3. rouge scoring
    '''

    for i in range(start, stop):

        ranked_sentences = run_textrank_model(df.iloc[i]['bert_text'])
        prediction = run_pegasus_large_model(ranked_sentences)
        reference = df.iloc[i]['plot_outline']
        rouge_scores = calculate_rouge_scores(prediction, reference)

        # add rouge scores and summary to a dictionary
        title = df.iloc[i]['title']
        rouge_dict[title] = rouge_scores
        summary_dict[title] = prediction

    return rouge_dict, summary_dict

In [ ]:
def execute_two_stage_text_t5_model(start, stop, rouge_dict, summary_dict, df):
    '''
    this function executes 
    1. textrank
    2. abstractive summarization (long t5 models)
    3. rouge scoring
    '''

    for i in range(start, stop):

        ranked_sentences = run_longt5_model(df.iloc[i]['bert_text'])
        prediction = run_pegasus_large_model(ranked_sentences)
        reference = df.iloc[i]['plot_outline']
        rouge_scores = calculate_rouge_scores(prediction, reference)

        # add rouge scores and summary to a dictionary
        title = df.iloc[i]['title']
        rouge_dict[title] = rouge_scores
        summary_dict[title] = prediction

    return rouge_dict, summary_dict

In [ ]:
def combine_rouge_and_summary_results(rouge_dict, summary_dict, abstractive_model_name):
    '''
    create a single dataframe that combines results from a given model
    '''

    # add rouge results to a pandas dataframe
    rouge_df = pd.DataFrame(rouge_dict).T

    # add summaries to a pandas series
    summary_series = pd.Series(summary_dict)

    # merge dataframe and series
    combined_results_df = rouge_df.merge(summary_series.rename(f'{abstractive_model_name}_summary'), left_index=True, right_index=True)

    return combined_results_df

# Data

### Import

#### Wikipedia references

In [ ]:
try:
    wikipedia_summary = pd.read_csv('/content/drive/MyDrive/Wikipedia_movie_meta_data.csv')
    wikipedia_summary.sort_values(by=['Title'])
    display(wikipedia_summary.head())
except FileNotFoundError:
    print('wikipedia_summary not found')

,Age Restrict,Akas,Awards,Budget,Cast,Cast1,Casting Directors,Countries,Director,Directors,...,Title1,Wiki Page,Writers,Year,Count Matches,F1,Imdb User Rating,Imdbid,Metascore,Number Of Imdb User Votes
0,"Argentina:Atp, Australia:M, Brazil:Livre, Cana...",Spider-Man 2: The IMAX Experience (United Stat...,"Oscar 2005, BAFTA Film Award 2005, Movies for ...","$200,000,000 (estimated)","Tobey Maguire, Kirsten Dunst, James Franco, Al...","Tobey Maguire, Kirsten Dunst, James Franco, Al...",Dianne Crittenden,United States,Sam Raimi,Sam Raimi,...,Spider-Man 2,https://en.wikipedia.org/wiki/Spider-Man_2,"Stan Lee, Steve Ditko, Alfred Gough, Miles Mil...",2004,1,0.0,7,316654,83,538496
1,NaN,NaN,NaN,NaN,"Hanna Lee, Bernard Siegel, Templar Saxe, Nita ...",NaN,NaN,United States,NaN,Lester Park,...,NaN,NaN,"Willard King Bradley, Willard King Bradley",1923,0,1.0,-1,164167,-1,-1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2012,0,2.0,-1,2761612,-1,-1
3,NaN,Shining The Light (United States),NaN,NaN,"Denise Jaxon, Alan Santana",NaN,NaN,United States,NaN,NaN,...,NaN,NaN,"Kevin Karp, Alan Santana",2019,0,3.0,-1,9106126,-1,-1
4,NaN,Smoke Gets in Your Eyes (United States),NaN,NaN,"William Conrad, Joe Penny, Alan Campbell, Mark...",NaN,"Sally Powers, Susan Scudder",United States,NaN,Harvey S. Laidman,...,NaN,NaN,"Dean Hargrove, Joel Steiger, Douglas Stefen Bo...",1987,0,4.0,7,614236,-1,19


#### BERT annotations

In [ ]:
bert_annotations_file_path = '/content/drive/MyDrive/W266_Movie_Data/BERT_annotations/'

try:
    all_files_bert = os.listdir(bert_annotations_file_path)
    print(f'{len(all_files_bert)} files found')
    print(all_files_bert)
except FileNotFoundError:
    print('bert_annotations not found')

1998 files found
['One Eight Seven_0118531_anno.txt', 'Mystic River_0327056_anno.txt', 'Moon_1182345_anno.txt', 'Mr Mark Fenton_1282592_anno.txt', 'Mr Destiny_0100201_anno.txt', 'Napoleon and Samantha_0068990_anno.txt', 'Only Lovers Left Alive_1714915_anno.txt', 'Mulholland Dr_1619856_anno.txt', 'Novitiate_4513316_anno.txt', 'Missing_0084335_anno.txt', 'Monsters University_1453405_anno.txt', 'Network_0074958_anno.txt', 'Mission Impossible_0117060_anno.txt', 'Okja_3967856_anno.txt', 'Next_0435705_anno.txt', 'Music of the Heart_0166943_anno.txt', 'Mistress America_2872462_anno.txt', 'Moonrise Kingdom_1748122_anno.txt', 'Norma Rae_0079638_anno.txt', 'Monster s Ball_0285742_anno.txt', 'Moulin Rouge_0203009_anno.txt', 'Ordinary People_0081283_anno.txt', 'Noah_1959490_anno.txt', 'Night of the Living Dead_0063350_anno.txt', 'Monsters Inc_0198781_anno.txt', 'Mr Brooks_0780571_anno.txt', 'Moneyball_1210166_anno.txt', 'Olympus Has Fallen_2302755_anno.txt', 'Nothing But a Man_0058414_anno.txt', '

#### Raw text lemmas

In [ ]:
raw_text_lemmas_file_path = '/content/drive/MyDrive/W266_Movie_Data/raw_text_lemmas/raw_text_lemmas/'

try:
    all_files_lemmas = sorted(os.listdir(raw_text_lemmas_file_path))
    print(f'{len(all_files_lemmas)} files found')
    print(all_files_lemmas)
except FileNotFoundError:
    print('raw_text_lemmas not found')

2941 files found
['10 Cloverfield Lane_1179933_lemmas.txt', '10 Things I Hate About You_0147800_lemmas.txt', '101 Days of 101 Dalmatians_0249328_lemmas.txt', '12 Angry Men_0118528_lemmas.txt', '12 Monkeys_0114746_lemmas.txt', '12 Years a Slave_2024544_lemmas.txt', '127 Hours_1542344_lemmas.txt', '13 13 13_2991516_lemmas.txt', '1408_0450385_lemmas.txt', '1492 Conquest of Paradise_0103594_lemmas.txt', '15 Minutes_0179626_lemmas.txt', '16 Blocks_0450232_lemmas.txt', '17 Again_0974661_lemmas.txt', '1917_8579674_lemmas.txt', '1976 l lection du Parti qu b cois_11327984_lemmas.txt', '2 Broke Girls_1845307_lemmas.txt', '2 Guns a Grenade a Pizza Guy_1871996_lemmas.txt', '2001 A Space Road Odyssey_0288910_lemmas.txt', '2010 The Odyssey Continues_0235153_lemmas.txt', '2012_1190080_lemmas.txt', '20th Century Women_4385888_lemmas.txt', '21 Grams_0315733_lemmas.txt', '21 Jump Street_1232829_lemmas.txt', '22 July_7280898_lemmas.txt', '25th Hour_0307901_lemmas.txt', '28 Days Later_0289043_lemmas.txt',

## Format data

#### Wikipedia references

In [ ]:
# Wikipedia
wiki_df = wikipedia_summary[['Title','Imdb Id','Plot Outline']]

# create imdb_id that is of type string
wiki_df['imdb_id_str'] = wiki_df['Imdb Id'].astype(str).str.split('.').str[0]

# rename columns
wiki_df = wiki_df.rename(columns={'Title': 'title',
                                  'imdb_id_str': 'imdb_id',
                                  'Plot Outline': 'plot_outline'})

# only keep certain columns
wiki_df = wiki_df[['title', 'imdb_id', 'plot_outline']]

# only keep movies that have plot outlines
wiki_df = wiki_df[wiki_df['plot_outline'].notna()]

# display
display(wiki_df.head())
print(len(wiki_df))

<ipython-input-26-de2efe5314cb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wiki_df['imdb_id_str'] = wiki_df['Imdb Id'].astype(str).str.split('.').str[0]


,title,imdb_id,plot_outline
0,Spider-Man 2,316654,Peter Parker is an unhappy man: after two year...
3,Shining The Light,9106126,Shining The Light will have a panel of six you...
4,Smoke Gets in Your Eyes,614236,A woman who was assaulted and left blind leave...
5,Solstice,473267,Six months after the suicide of her twin siste...
6,Something Wicked This Way Comes,86336,"In Green Town, Illinois, the twelve year-old b..."


2498


#### BERT annotations

In [ ]:
# BERT
bert_df = pd.DataFrame(all_files_bert, columns=['filename'])

# get title from filename
bert_df['title'] = bert_df['filename'].str.split('_').str[0]

# get imdb id from filename
bert_df['imdb_id'] = bert_df['filename'].str.split('_').str[1]

# create filepath to script
bert_df['bert_data'] = bert_annotations_file_path + bert_df['filename']

# display
display(bert_df.head())
print(len(bert_df))

,filename,title,imdb_id,bert_data
0,One Eight Seven_0118531_anno.txt,One Eight Seven,0118531,/content/drive/MyDrive/W266_Movie_Data/BERT_an...
1,Mystic River_0327056_anno.txt,Mystic River,0327056,/content/drive/MyDrive/W266_Movie_Data/BERT_an...
2,Moon_1182345_anno.txt,Moon,1182345,/content/drive/MyDrive/W266_Movie_Data/BERT_an...
3,Mr Mark Fenton_1282592_anno.txt,Mr Mark Fenton,1282592,/content/drive/MyDrive/W266_Movie_Data/BERT_an...
4,Mr Destiny_0100201_anno.txt,Mr Destiny,0100201,/content/drive/MyDrive/W266_Movie_Data/BERT_an...


1998


#### Raw text lemmas

In [ ]:
# Lemma
lemma_df = pd.DataFrame(all_files_lemmas, columns=['filename'])

# get title from filename
lemma_df['title'] = lemma_df['filename'].str.split('_').str[0]

# get imdb id from filename
lemma_df['imdb_id'] = lemma_df['filename'].str.split('_').str[1]

# create filepath to script
lemma_df['lemma_data'] = raw_text_lemmas_file_path + lemma_df['filename']

# display
display(lemma_df.head())
print(len(lemma_df))

,filename,title,imdb_id,lemma_data
0,10 Cloverfield Lane_1179933_lemmas.txt,10 Cloverfield Lane,1179933,/content/drive/MyDrive/W266_Movie_Data/raw_tex...
1,10 Things I Hate About You_0147800_lemmas.txt,10 Things I Hate About You,0147800,/content/drive/MyDrive/W266_Movie_Data/raw_tex...
2,101 Days of 101 Dalmatians_0249328_lemmas.txt,101 Days of 101 Dalmatians,0249328,/content/drive/MyDrive/W266_Movie_Data/raw_tex...
3,12 Angry Men_0118528_lemmas.txt,12 Angry Men,0118528,/content/drive/MyDrive/W266_Movie_Data/raw_tex...
4,12 Monkeys_0114746_lemmas.txt,12 Monkeys,0114746,/content/drive/MyDrive/W266_Movie_Data/raw_tex...


2941


### Combine DataFrames
Combination is done based on imdb_id


In [ ]:
# merge bert and lemma dataframes
data_df = pd.merge(bert_df, lemma_df, how='outer', on='imdb_id')

# rename columns
data_df = data_df.rename(columns={'filename_x': 'bert_filename',
                                  'filename_y': 'lemma_filename',
                                  'title_x': 'title'})

# reorder columns
data_df = data_df[['title', 'imdb_id', 'bert_data', 'lemma_data']]

# display
display(data_df.head())
print(len(data_df))

,title,imdb_id,bert_data,lemma_data
0,One Eight Seven,0118531,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...
1,Mystic River,0327056,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...
2,Moon,1182345,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...
3,Mr Mark Fenton,1282592,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...
4,Mr Destiny,0100201,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...


2941


In [ ]:
# merge bert + lemma (data_df) with wikipedia dataframe
final_df = pd.merge(data_df, wiki_df, how='outer', on='imdb_id')

# drop rows that have NaN in any of the columns
final_df = final_df.dropna(how='any')

# drop movies that have duplicate imdb_ids
final_df = final_df.drop_duplicates(subset='imdb_id', keep='first')

# rename columns
final_df = final_df.rename(columns={'title_x': 'title'})

# reorder columns
final_df = final_df[['title', 'imdb_id', 'bert_data', 'lemma_data', 'plot_outline']]

# reindex rows
final_df.reset_index(drop=True, inplace=True)

# display
display(final_df.head())
print(len(final_df))

,title,imdb_id,bert_data,lemma_data,plot_outline
0,Moon,1182345,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,Sam Bell has a three year contract to work for...
1,Only Lovers Left Alive,1714915,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"Adam (Tom Hiddleston), an underground musician..."
2,Mulholland Dr,1619856,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,Yearning to spread her wings and make a name f...
3,Novitiate,4513316,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"1964. Cathleen Harris, in her late teens, has ..."
4,Monsters University,1453405,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"Mike Wazowski (Billy Crystal) and James P. ""Su..."


565


There are 565 examples, but only keep 500 of them with random state 266 (for DATASCI 266 🙂)

In [ ]:
# keep only 500 samples
samples_to_keep = 500
pct_samples_to_keep = samples_to_keep / len(final_df)

# assign final_df to df for ease of use
df = final_df.sample(frac=pct_samples_to_keep, random_state=266)

# display
display(df.head())
print(len(df))

,title,imdb_id,bert_data,lemma_data,plot_outline
341,Buried,1462758,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"Waking groggy in pitch darkness, Paul Conroy, ..."
452,I Smile Back,3640682,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,Laney Brooks does bad things. Married with kid...
119,The Edge of Seventeen,1878870,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"Everyone knows that growing up is hard, and li..."
131,The Company Men,1172991,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,When the GTX Corporation must cut jobs to impr...
247,Upgrade,6499752,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"Grey's a stay-at-home mechanic, whose wife is ..."


500


In [ ]:
# double check for duplicates (vc = value counts)
title_vc = df['title'].value_counts()
print(title_vc[title_vc > 1])

imdb_id_vc = df['imdb_id'].value_counts()
print(imdb_id_vc[imdb_id_vc > 1])

Series([], Name: title, dtype: int64)
Series([], Name: imdb_id, dtype: int64)


In [ ]:
# only add the bert dialog and text to the trimmed data to save on resources and time

# get bert dialog
df['bert_dialog'] = df.apply(lambda x: concatenate_dialog(x['bert_data']), axis=1)

# get bert text
df['bert_text'] = df.apply(lambda x: concatenate_text(x['bert_data']), axis=1)

In [ ]:
# train / val / test split
#    70 / 10  / 20 

train_pct = int(0.7 * len(df))
val_pct = int(0.8 * len(df))

# the split is done using a numpy solution
# first shuffle all of the samples at a random state
# then split into the percentages above
train_df, val_df, test_df = np.split(df.sample(frac=1, random_state=266), [train_pct, val_pct])

print(len(train_df))
print(len(val_df))
print(len(test_df))

350
50
100


In [ ]:
# check dataframes

# train
display(train_df.head())

# val
display(val_df.head())

# test
display(test_df.head())

,title,imdb_id,bert_data,lemma_data,plot_outline,bert_dialog,bert_text
221,Three Billboards Outside Ebbing Missouri,5027774,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"THREE BILLBOARDS OUTSIDE EBBING, MISSOURI is a...","by\n Martin McDonagh\n You Red Welby?\n Yes, ...","MILDRED HAYES, a woman in her early 50's, dri..."
346,Candle to Water,2387411,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,To present five different contemporary stories...,\n Written by\n Nick Green\n Shooting draft (...,"The car is stuck in rush-hour traffic, which ..."
277,1917,8579674,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"April 1917, the Western Front. Two British sol...",1917\n Written by\n Sam Mendes\n &\n Krysty W...,The following script takes place in real time...
420,Friends with Benefits,1632708,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,Jamie Rellis (Mila Kunis) is a New York City h...,No. Not even close.\n I know. I'll be there.\...,He almost steps on a dog sleeping on the floo...
315,Anonymous,1521197,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"Edward De Vere, Earl of Oxford, is presented a...","\n Written by\n John Orloff\n up, etc...\n th...",TITLES BEGIN over the SOUNDS of city traffic....


,title,imdb_id,bert_data,lemma_data,plot_outline,bert_dialog,bert_text
6,Mistress America,2872462,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"Tracy, a lonely college freshman in New York, ...",Written by\n Noah Baumbach and Greta Gerwig\n...,much fun to agree with her.\n A dark room. We...
428,Fargo,2802850,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"The all new ""true crime"" case of Fargo's new c...",\n a screenplay by\n Ethan Coen\n and\n Joel ...,The following text fades in over black This i...
432,Gangs of Wasseypur,1954470,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,Shahid Khan is exiled after impersonating the ...,"Ha se?\n wer erftrst Wat eiiteat!\n Shia, SN ...",Sardar is at the door with young Definite.\n ...
379,Cloud Atlas,1371111,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,Everything is connected: an 1849 diary of an o...,A Film By\n A. Wachowski\n L. Wachowski\n T. ...,"On motes like meteors; dancing, streaking par..."
240,Wall Street Money Never Sleeps,1027718,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,As the global economy teeters on the brink of ...,"\n \n Written by\n Allan Loeb\n ""When busines...",Onto which is written in white letters...\n T...


,title,imdb_id,bert_data,lemma_data,plot_outline,bert_dialog,bert_text
271,21 Jump Street,1232829,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"In 2005, Schmidt and Jenko are high school stu...","By Oren Uziel\n 4/23/13\n Previously, on 21 J...",Long-haired JENKO watches Slim-Shady SCHMIDT ...
490,If Beale Street Could Talk,7125860,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,African-American teen sweethearts Fonny and Ti...,Written by\n Barry Jenkins\n Adapted from the...,IF BEALE STREET COULD TALK\n +.+.ON a young m...
476,J Edgar,1616195,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,Biopic of J. Edgar Hoover (Leonardo DiCaprio) ...,Screenplay by\n Dustin Lance Black\n 09-29-20...,"CLOSE ON STRAW HAT, his HALF SMOKED CIGAR and..."
279,A Simple Favor,7040874,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"Stephanie, a widowed mother, who also runs a c...","Written by\n Jessica Sharzer\n June 29th, 201...","Revisions by Jessica Sharzer, Peter Craig and..."
15,Molly s Game,4209788,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"Molly Bloom, a beautiful young Olympic-class s...",screenplay by\n Aaron Sorkin\n Based on the b...,But we don’t know what we’re looking at yet. ...


In [ ]:
# export final dataframes

# all 500 samples (train/val/test BEFORE split)
df.to_csv('/content/complete_wiki_bert_lemma.csv')

# train split
train_df.to_csv('/content/train_wiki_bert_lemma.csv')

# val split
val_df.to_csv('/content/val_wiki_bert_lemma.csv')

# test split
test_df.to_csv('/content/test_wiki_bert_lemma.csv')

In [ ]:
# use random state 266 for taking only 30 samples from the training data set
subset_train_df = train_df.sample(n=30, axis=0, random_state=266)

In [ ]:
# check for NaNs
subset_train_df.isna().sum().sum()

0

In [ ]:
# save results to a csv
subset_train_df.to_csv('/content/subset_train_df.csv')
# files.download('/content/subset_train_df.csv') 

# More Exploratory Data Analysis
Since there are too many tokens in any given movie script and not enough computational resources, selectively choose which tokens are used. Thankfully, BERT annotation data splits the script into 4 main elements: *dialog, text, scene_heading, and speaker_heading*

<br>

The intution behind the below code is to compare ROUGE scores between only concatenating dialog and only concatening text from movie scripts.

The steps are broken down further:

1.   Concatenate script element (dialog or text)
2.   Run TextRank algorithm to score sentences (extract only top X sentences)
3.   Run abstractive summary model
4.   Compute ROUGE (Recall-Oriented Understudy for Gisting Evaluation) score



Concatenated bert_dialog

In [ ]:
ranked_sentences = run_textrank_model(train_df.iloc[0]['bert_dialog'])
calculate_rouge_scores(ranked_sentences, train_df.iloc[0]['plot_outline'])

1240


{'rouge1': 0.08264462809917356,
 'rouge2': 0.0,
 'rougeL': 0.03856749311294766,
 'rougeLsum': 0.06978879706152433}

In [ ]:
ranked_sentences = run_textrank_model(train_df.iloc[1]['bert_dialog'])
calculate_rouge_scores(ranked_sentences, train_df.iloc[1]['plot_outline'])

1327


{'rouge1': 0.11272141706924314,
 'rouge2': 0.012924071082390952,
 'rougeL': 0.0644122383252818,
 'rougeLsum': 0.10305958132045087}

In [ ]:
ranked_sentences = run_textrank_model(train_df.iloc[2]['bert_dialog'])
calculate_rouge_scores(ranked_sentences, train_df.iloc[2]['title'])

1249


{'rouge1': 0.20979020979020976,
 'rouge2': 0.02112676056338028,
 'rougeL': 0.10489510489510488,
 'rougeLsum': 0.13986013986013984}

In [ ]:
# export an example to csv
example_df = pd.DataFrame(data={'title': train_df.iloc[2]['title'],
                                'ranked_sentences': ranked_sentences,
                                'plot_outline': train_df.iloc[2]['plot_outline']},
                          index=[0])
example_df.to_csv('/content/1917_top_50_ranked.csv')

Concatenated bert_text

In [ ]:
ranked_sentences = run_textrank_model(train_df.iloc[0]['bert_text'])
calculate_rouge_scores(ranked_sentences, train_df.iloc[0]['plot_outline'])

539


{'rouge1': 0.05534105534105534,
 'rouge2': 0.007731958762886597,
 'rougeL': 0.03732303732303732,
 'rougeLsum': 0.0501930501930502}

In [ ]:
ranked_sentences = run_textrank_model(train_df.iloc[1]['bert_text'])
calculate_rouge_scores(ranked_sentences, train_df.iloc[1]['plot_outline'])

1345


{'rouge1': 0.07547169811320754,
 'rouge2': 0.007566204287515764,
 'rougeL': 0.05534591194968553,
 'rougeLsum': 0.06792452830188679}

In [ ]:
ranked_sentences = run_textrank_model(train_df.iloc[2]['bert_text'])
calculate_rouge_scores(ranked_sentences, train_df.iloc[2]['plot_outline'])

1757


{'rouge1': 0.055315471045808126,
 'rouge2': 0.0069264069264069255,
 'rougeL': 0.04148660328435609,
 'rougeLsum': 0.05185825410544512}

As you can see from the examples above, using bert_dialog shows better summarizations than using bert_text

# Baseline Models (Raw text lemmas)
All baseline models will be run with 30 sample movie scripts from random seed 266

## Pegasus Short

In [ ]:
pegasus_short_lemma_rouge_dict = {}
pegasus_short_lemma_summary_dict = {}

for i in range(30):
    
    raw_lemma = subset_train_df.iloc[i]['lemma_data']
    with open(raw_lemma, 'r') as file:
        text_data = file.read().replace('\n', '')

    pegasus_short_summary = run_pegasus_short_model(text_data)
    pegasus_short_rouge_scores = calculate_rouge_scores(pegasus_short_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    pegasus_short_lemma_summary_dict[title] = pegasus_short_summary
    pegasus_short_lemma_rouge_dict[title] = pegasus_short_rouge_scores

print(pegasus_short_lemma_rouge_dict)
print(pegasus_short_lemma_summary_dict)

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('A video of a police chase in the Republic of Ireland in which a car ploughed '
 'into a patrol car, killing one officer and injuring three others, has been '
 'released by the Irish National Police (INPS) and the Garda Sochna (Irish '
 "police) on the eve of the Republic's independence day.... and it's all "
 'captured on a helmet-mounted camera, with the help of a thermal imaging '
 'camera and a thermal camera system.')


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Batman v Superman: Dawn of Justice director: Zack Snyder starring: Ben '
 'Affleck, Henry Cavill, Chris Evans, Gal Gadot, Ray Stevenson, Scoot McNairy, '
 'Jeremy Irons, JK Simmons, Rosario Dawson, Benicio del Toro, Michael Keaton, '
 'Ben Kingsley, Holly Hunter, Michael Sheen, and director: Frank Miller '
 'starring: Henry Cavill as Batman, Ben Affleck as Bruce Wayne, Gal Affleck as '
 "Batman's alter-ego, Robin, Chris Pine as Wonder Woman, Amy Adams as Lois "
 "Lane, Michael Shannon as John Wayne's wife, Talia Shire as Robin's "
 'ex-girlfriend, Rachel McAdams as Wonder')


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

('Watch the video for the song " shall be the light of the world" by the band '
 'The Flaming Lips on their official website: www.the Flaming Lips Facebook '
 'page: '
 'https://www.facebook.com/pages/The-Flaming-Lips-on-LLC-LLC/3311936574574?ref=ts '
 "- by the Flaming Lips - with permission from the band's record label, "
 'Interscope Records - on their website: http://bit.ly/2u9w5sr')


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("Jane the Virgin cast member ryan hansen explains what it's like to play the "
 'title role in the new drama about a group of misfit high school students who '
 "team up to solve a mystery in the final episodes of the series' first "
 "season. smh - i'm not a fan of the show, but i'll give it a shot - i love "
 "ryan - he's a great guy - i think he'll do a great job - i want to work with "
 "him again - i don't know what else to say - i just want to see him on the "
 'show - i hope he')


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Pacific lexicon by Travis beacham preface : translate a supplementary guide '
 'to pertinent term jargon and terminology for sailors, airmen, Marines, '
 'soldiers, sailors, and others who serve, live, work, or train in the Pacific '
 'Ocean and its surrounding waters.) by quasi pacific lexicon byTravis beacham '
 'preamble : Translate a supplementary Guide to Relevant Terms and Terminology '
 'for Sailors, Airmen, Marine, Soldiers, sailors and others Who Serve, Live, '
 'Work, or Train in thePacific Ocean and Its Surrounding Waters.')
{'The Guard': {'rouge1': 0.19672131147540983, 'rouge2': 0.016666666666666663, 'rougeL': 0.11475409836065574, 'rougeLsum': 0.11475409836065574}, 'Batman Year One': {'rouge1': 0.08653846153846154, 'rouge2': 0.009708737864077669, 'rougeL': 0.04807692307692308, 'rougeLsum': 0.04807692307692308}, 'Inherent Vice': {'rouge1': 0.2962962962962963, 'rouge2': 0.07518796992481203, 'rougeL': 0.14814814814814814, 'rougeLsum': 0.14814814814814814}, 'Colombiana': {'ro

In [ ]:
pegasus_short_lemma_results = combine_rouge_and_summary_results(pegasus_short_lemma_rouge_dict, pegasus_short_lemma_summary_dict, 'pegasus_short_lemma')
pegasus_short_lemma_results.head()

,rouge1,rouge2,rougeL,rougeLsum,pegasus_short_lemma_summary
The Guard,0.196721,0.016667,0.114754,0.114754,A video of a police chase in the Republic of I...
Batman Year One,0.086538,0.009709,0.048077,0.048077,Batman v Superman: Dawn of Justice director: Z...
Inherent Vice,0.296296,0.075188,0.148148,0.148148,Thomas Anderson's film adaptation of the novel...
Colombiana,0.220690,0.027972,0.151724,0.151724,Comic book writer Robert Mark kamen reveals th...
Barney s Version,0.133333,0.000000,0.116667,0.116667,mordecai richler's novel 'The Sense of an Endi...


In [ ]:
# save results to a csv
pegasus_short_lemma_results.to_csv('/content/pegasus_short_lemma_results.csv')
# files.download('/content/pegasus_short_lemma_results.csv') 

In [ ]:
# average rouge scores
pegasus_short_lemma_results.drop(columns=['pegasus_short_lemma_summary']).mean(axis=0)

rouge1       0.194040
rouge2       0.020054
rougeL       0.118927
rougeLsum    0.118927
dtype: float64

## Pegasus Long

In [ ]:
pegasus_long_lemma_rouge_dict = {}
pegasus_long_lemma_summary_dict = {}

for i in range(30):

    raw_lemma = subset_train_df.iloc[i]['lemma_data']
    with open(raw_lemma, 'r') as file:
        text_data = file.read().replace('\n', '')

    pegasus_long_summary = run_pegasus_long_model(text_data)
    pegasus_long_rouge_scores = calculate_rouge_scores(pegasus_long_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    pegasus_long_lemma_summary_dict[title] = pegasus_long_summary
    pegasus_long_lemma_rouge_dict[title] = pegasus_long_rouge_scores

print(pegasus_long_lemma_rouge_dict)
print(pegasus_long_lemma_summary_dict)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Final shoot draft contains all revision January 2010 reprisal film / element '
 'picture 21 mespil Road Dublin 4 Ireland .<n> dawn helicopter shot -- a red '
 'car speed through the barren landscape ., veer wildly , just as the sun be '
 'rise , glint of light break the darkness . int. red car - dawn five young '
 'men be pass a whiskey bottle around , ext . country road - dawn a garda '
 'police car be park in the road .')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('batman year one screenplay by frank miller fade in .<n> ext . gotham CITY , '
 'night lightning rip across a jet-black sky ..<n>Thunder crash . lightning '
 'cast gotham City in stark relief . siren wail . car alarm scream . lighting '
 'silhouettes a menace police helicopter ., roaring downward like a monster '
 'insect . Lightning ... spark? shower. wild . reveal : int. dark area - night '
 "a sweat , torment sleeper '' bruce wayne .")


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

("'Seth' gets out of his car in the parking lot of a restaurant and goes to "
 'the pick-up counter .<n>He waits for the cashier to look up, and then drop '
 'he tip in the ja.<n>Video was taken at a restaurant in Los Angeles, '
 "California, U.S.<n>Seth's name is Seth and he is a bartender at a bar called "
 'Third Man Tavern. He lives with his girlfriend, Hailey Hill, and their two '
 'children, aged two and four. He has a son, Seth Jr., who is three years old.')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('ryan hansen solves crime on television .<n>Ryan hansen is played by rawson '
 'Marshall thurber episode # 102 February lst, 2017.<n>Read the full episode '
 "on steal chinese internet and that 's cool too.<n>Follow Ryan hansen on "
 'Twitter @ryanhansen and on Facebook @ryanHansen . GRAPHIC CONTENT MAY OFFEND '
 'READERS: !--WARNING: GRAPHIC IMAGES -- .')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Pacific rim .<n> lexicon ( a supplementary guide to pertinent term and '
 'jargon ... ).<n>Travis beacham is a marine engineer and author of marine '
 'technology books.<n>His company, Travis Beacham & Co., conducts marine '
 'research and development projects in the U.S. and overseas.<n>Travis '
 'Beacham’s company also conducts marine exploration and testing operations in '
 'the Pacific Ocean and the Indian Ocean.<n>He is the author of the book '
 'Pacific Rim: The Official Encyclopedia of the United States of America, '
 'published by Simon & Schuster.')
{'The Guard': {'rouge1': 0.1016949152542373, 'rouge2': 0.017241379310344827, 'rougeL': 0.1016949152542373, 'rougeLsum': 0.1016949152542373}, 'Batman Year One': {'rouge1': 0.13186813186813187, 'rouge2': 0.022222222222222223, 'rougeL': 0.054945054945054944, 'rougeLsum': 0.054945054945054944}, 'Inherent Vice': {'rouge1': 0.15748031496062992, 'rouge2': 0.0, 'rougeL': 0.11023622047244094, 'rougeLsum': 0.11023622047244094}, 'Colombiana

In [ ]:
pegasus_long_lemma_results = combine_rouge_and_summary_results(pegasus_long_lemma_rouge_dict, pegasus_long_lemma_summary_dict, 'pegasus_long_lemma')
pegasus_long_lemma_results.head()

,rouge1,rouge2,rougeL,rougeLsum,pegasus_long_lemma_summary
The Guard,0.101695,0.017241,0.101695,0.101695,Final shoot draft contains all revision Januar...
Batman Year One,0.131868,0.022222,0.054945,0.054945,batman year one screenplay by frank miller fad...
Inherent Vice,0.157480,0.000000,0.110236,0.110236,This script is the confidential and proprietar...
Colombiana,0.164179,0.000000,0.119403,0.119403,10 year old girl turns page of a notebook to r...
Barney s Version,0.196970,0.015385,0.106061,0.106061,Michael Konyves's version of the novel by mord...


In [ ]:
# save results to a csv
pegasus_long_lemma_results.to_csv('/content/pegasus_long_lemma_results.csv')
# files.download('/content/pegasus_long_lemma_results.csv') 

In [ ]:
# average rouge scores
pegasus_long_lemma_results.drop(columns=['pegasus_long_lemma_summary']).mean(axis=0)

rouge1       0.173448
rouge2       0.012663
rougeL       0.105938
rougeLsum    0.105938
dtype: float64

## Pegasus Large

In [ ]:
pegasus_large_lemma_rouge_dict = {}
pegasus_large_lemma_summary_dict = {}

for i in range(30):

    raw_lemma = subset_train_df.iloc[i]['lemma_data']
    with open(raw_lemma, 'r') as file:
        text_data = file.read().replace('\n', '')

    pegasus_large_summary = run_pegasus_large_model(text_data)
    pegasus_large_rouge_scores = calculate_rouge_scores(pegasus_large_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    pegasus_large_lemma_summary_dict[title] = pegasus_large_summary
    pegasus_large_lemma_rouge_dict[title] = pegasus_large_rouge_scores

print(pegasus_large_lemma_rouge_dict)
print(pegasus_large_lemma_summary_dict)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('connemara - dawn the red car zoom along at breakneck speed -- ext . country '
 'road - dawn eventually shoot past boyle in he police car -- int . garda car '
 '- dawn boyle barely react -- suddenly there come the sound of screeching '
 'brake , and the boom of a high - impact car crash -- boyle unhurriedly start '
 'the car . bridge - sunrise the Red car have slam stra t n tnn.')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("Close batman year one screenplay by frank miller fade in . bruce wayne's "
 'apartment - night awoken from he nightmare by the sound of he own scream '
 'silhouettes , disorient , bruce go to the sink and splashes cold wate ? '
 "bruce's bedroom - night woken from his nightmare by his own scream , his "
 'disoriental , he go to his sink and splash cold , . .close batman year 1 '
 'screenplay by Frank Miller fade .')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

('eth pay and be about to drop a dollar in the tip jar , but notice the '
 'cashier be not look . he wait for the cashier to look up , and then drop he '
 'tip in the ja Copyright  2018 by craig robinson , all rights reserved , '
 'craig.robinson@gmail.com , www.craigrobinson.com and '
 'www.youtube.com/craigrobinston , All rights reserved.')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('runyon canyon - day ryan hansen , shirtless and lightly perspire ( never '
 'sweat ) , huff and puff he way up this iconic " hike " trail in the '
 "Hollywood hill , all the while talk directly into he iphone 's periscope app "
 '. YouTube Red in da hiz-ouse ! ryan ( cont would ) ytr guy say there be a " '
 'less than normal " viewership loss from this crazy viral twerk grandma video '
 'that they link we to .')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('brain-to-brain interface ( see pon or drift ) * conn-pod . ( slang ) a joint '
 'vision share by jaeger crew member while directly link via the pon ; trigger '
 'by subconscious stimulus , a generally involuntary but seldom debilite '
 'phenomenon + drivesuit . The jaeger / pilot interface suit design to monitor '
 'vital sign and to translate nerve signal to piloting input * ghost-drift. ( '
 'a kind of quasi-telepathic vision share ( by party with a pre-exist pon '
 'connection ) without the aid of a hard neural interface ; commonly trigger '
 'by stress , physical proximity , or coincide rem sleep pattern + headspace .')
{'The Guard': {'rouge1': 0.10909090909090909, 'rouge2': 0.0, 'rougeL': 0.07272727272727274, 'rougeLsum': 0.07272727272727274}, 'Batman Year One': {'rouge1': 0.1925133689839572, 'rouge2': 0.02162162162162162, 'rougeL': 0.10695187165775401, 'rougeLsum': 0.10695187165775401}, 'Inherent Vice': {'rouge1': 0.0847457627118644, 'rouge2': 0.017241379310344824, 'rougeL': 0.

In [ ]:
pegasus_large_lemma_results = combine_rouge_and_summary_results(pegasus_large_lemma_rouge_dict, pegasus_large_lemma_summary_dict, 'pegasus_large_lemma')
pegasus_large_lemma_results.head()

,rouge1,rouge2,rougeL,rougeLsum,pegasus_large_lemma_summary
The Guard,0.109091,0.000000,0.072727,0.072727,connemara - dawn the red car zoom along at bre...
Batman Year One,0.192513,0.021622,0.106952,0.106952,Close batman year one screenplay by frank mill...
Inherent Vice,0.084746,0.017241,0.050847,0.050847,'' 2013 4000 Warner boulevard warner bros . ba...
Colombiana,0.183206,0.000000,0.122137,0.122137,colombiana written by Robert Mark kamen &amp ;...
Barney s Version,0.184874,0.000000,0.100840,0.100840,den - barney and miriam apartment - night a tu...


In [ ]:
# save results to a csv
pegasus_large_lemma_results.to_csv('/content/pegasus_large_lemma_results.csv')
# files.download('/content/pegasus_large_lemma_results.csv') 

In [ ]:
# average rouge scores
pegasus_large_lemma_results.drop(columns=['pegasus_large_lemma_summary']).mean(axis=0)

rouge1       0.162353
rouge2       0.011145
rougeL       0.101918
rougeLsum    0.101918
dtype: float64

## LongT5

In [ ]:
longt5_lemma_rouge_dict = {}
longt5_lemma_summary_dict = {}

for i in range(30):

    raw_lemma = subset_train_df.iloc[i]['lemma_data']
    with open(raw_lemma, 'r') as file:
        text_data = file.read().replace('\n', '')

    longt5_summary = run_longt5_model(text_data)
    longt5_rouge_scores = calculate_rouge_scores(longt5_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    longt5_lemma_summary_dict[title] = longt5_summary
    longt5_lemma_rouge_dict[title] = longt5_rouge_scores

print(longt5_lemma_rouge_dict)
print(longt5_lemma_summary_dict)

("John Michaelael Mcdonagh's final shoot draft is a short scene set in the "
 'Irish countryside at dawn. A red car speeds past a police station and then '
 "crashes into a guard's car. Boyle reacts violently to the sound of a crash "
 'and jumps out of the car. Then, as the car speeds away, it lands on a cliff. '
 'This is one of the most famous scenes in the play.')
('"Batman Year One" is a flashback to the year of batman, Frank Miller\'s '
 'first play. It opens with a thunder crash and lightning. A police helicopter '
 'roars down like a giant insect. Int . Night a sweaty, tormented sleeper '
 'Bruce Wayne wakes up from his nightmare and screams in terror. He goes into '
 'the sink and gets splashed cold.')
('Paul Thomas Anderson\'s "Inheritance" is a screenplay based on the novel of '
 'Thomas Pynchon. It was written by Thomas Pynchon, and it is the property of '
 'Warner Bros. The story begins with a young woman walking up the alley in '
 'front of the apartment where she always 

In [ ]:
longt5_lemma_results = combine_rouge_and_summary_results(longt5_lemma_rouge_dict, longt5_lemma_summary_dict, 'longt5_lemma')
longt5_lemma_results.head()

,rouge1,rouge2,rougeL,rougeLsum,longt5_lemma_summary
The Guard,0.218487,0.034188,0.168067,0.168067,John Michaelael Mcdonagh's final shoot draft i...
Batman Year One,0.202247,0.011364,0.112360,0.112360,"""Batman Year One"" is a flashback to the year o..."
Inherent Vice,0.258993,0.043796,0.115108,0.115108,"Paul Thomas Anderson's ""Inheritance"" is a scre..."
Colombiana,0.242424,0.015385,0.166667,0.166667,This is a very short comic about a 10 year old...
Barney s Version,0.236220,0.000000,0.157480,0.157480,"The next morning, barney calls his wife and as..."


In [ ]:
# save results to a csv
longt5_lemma_results.to_csv('/content/longt5_lemma_results.csv')
# files.download('/content/longt5_lemma_results.csv') 

In [ ]:
# average rouge scores
longt5_lemma_results.drop(columns=['longt5_lemma_summary']).mean(axis=0)

rouge1       0.222060
rouge2       0.018561
rougeL       0.129684
rougeLsum    0.129684
dtype: float64

# Baseline Models (BERT - Dialog)
All baseline models will be run with 30 sample movie scripts from random seed 266

## Pegasus Short

---


Max 512 tokens for pretrained *'google/pegasus-xsum'* model

In [ ]:
pegasus_short_rouge_dict = {}
pegasus_short_summary_dict = {}

for i in range(30):
    pegasus_short_summary = run_pegasus_short_model(subset_train_df.iloc[i]['bert_dialog'])
    pegasus_short_rouge_scores = calculate_rouge_scores(pegasus_short_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    pegasus_short_summary_dict[title] = pegasus_short_summary
    pegasus_short_rouge_dict[title] = pegasus_short_rouge_scores

print(pegasus_short_rouge_dict)
print(pegasus_short_summary_dict)

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('All photographs  Element Pictures / Reprisal Films, courtesy of the Irish '
 'Film Board and the National Film Board of Ireland, with permission from the '
 'copyright holder, Mirtn  Mln.  The film is released in cinemas in the '
 'Republic of Ireland on 26 January, and on DVD and Blu-ray on 1 February '
 '2010.  Subscribe to the BBC News website: www.bbc.co.uk/news  Like us on '
 'Facebook: https://www.facebook.com/bbc News Follow us on Twitter: @BBCNews')


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("The following is an extract from Frank Miller's novel, The Man in the High "
 'Castle, in which the author reflects on a year in the life of his son, '
 'Spencer, who was killed in a car crash in New York City on Christmas Day, '
 "2014... maybe I'll kick you out of the house. Maybe you'll look fine, but "
 "I'm just not quite ready for that kind of work..maybe I'd like to take a "
 'break.')


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

('Jay Baruchel, Seth Rogen, and James Franco star in the comedy horror film '
 'The Book of Revelation, which opens in cinemas across the US on Friday, 22 '
 "March. (pleased to see you next time.) - Jay, I'm just here for an audition "
 "- I'd literally just get in and see if I can do it - I don't think I'll be "
 "able to do it, because I've got a job to do.")


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("It's been a while since I've written a post about a TV show, so I thought "
 'I\'d do it again for you all: "Hi, I\'m Jane D\' Poussin, and I have a new '
 "show on Netflix called Raw, and it's all about a Chinese detective called "
 "Redson, and he's the only detective in the world who can solve crimes in "
 "Chinese, so why don't you guys watch it?")


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('All terms used in this article are subject to change at any time, without '
 'notice, by the author or his or her AFFILIATES, including but not limited '
 'to, the BBC, the New York Times, the Washington Post, the Los Angeles Times, '
 'and other media outlets, and are copyrighted by their respective owners, '
 'unless otherwise expressly stated in this material, and may not be reprinted '
 'without the prior written consent of the author.., and all photographs are '
 'copyrighted..')
{'The Guard': {'rouge1': 0.128, 'rouge2': 0.016260162601626015, 'rougeL': 0.08, 'rougeLsum': 0.08}, 'Batman Year One': {'rouge1': 0.21105527638190955, 'rouge2': 0.01015228426395939, 'rougeL': 0.10050251256281408, 'rougeLsum': 0.10050251256281408}, 'Inherent Vice': {'rouge1': 0.22764227642276424, 'rouge2': 0.06611570247933884, 'rougeL': 0.13008130081300812, 'rougeLsum': 0.13008130081300812}, 'Colombiana': {'rouge1': 0.1678321678321678, 'rouge2': 0.014184397163120567, 'rougeL': 0.11188811188811189, 'rouge

In [ ]:
pegasus_short_results = combine_rouge_and_summary_results(pegasus_short_rouge_dict, pegasus_short_summary_dict, 'pegasus_short')
pegasus_short_results.head()

,rouge1,rouge2,rougeL,rougeLsum,pegasus_short_summary
The Guard,0.128000,0.016260,0.080000,0.080000,All photographs Element Pictures / Reprisal F...
Batman Year One,0.211055,0.010152,0.100503,0.100503,The following is an extract from Frank Miller'...
Inherent Vice,0.227642,0.066116,0.130081,0.130081,"The screenplay by Paul Thomas Anderson, based ..."
Colombiana,0.167832,0.014184,0.111888,0.111888,Cat in the Hat is a comic book series written ...
Barney s Version,0.158730,0.000000,0.111111,0.111111,"Watch the trailer for Barney's Version, Michae..."


In [ ]:
# save results to a csv
pegasus_short_results.to_csv('/content/pegasus_short_results.csv')
# files.download('/content/pegasus_short_results.csv') 

In [ ]:
# average rouge scores
pegasus_short_results.drop(columns=['pegasus_short_summary']).mean(axis=0)

rouge1       0.191756
rouge2       0.019610
rougeL       0.116721
rougeLsum    0.116721
dtype: float64

## Pegasus Long


---


Max 1024 tokens for pretrained *'google/pegasus-cnn_dailymail'* model

In [ ]:
pegasus_long_rouge_dict = {}
pegasus_long_summary_dict = {}

for i in range(30):
    pegasus_long_summary = run_pegasus_long_model(subset_train_df.iloc[i]['bert_dialog'])
    pegasus_long_rouge_scores = calculate_rouge_scores(pegasus_long_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    pegasus_long_summary_dict[title] = pegasus_long_summary
    pegasus_long_rouge_dict[title] = pegasus_long_rouge_scores

print(pegasus_long_rouge_dict)
print(pegasus_long_summary_dict)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Final Shooting Draft is a feature film by Reprisal Films and Element '
 'Pictures based on a short story by Stephen Fry .<n>The story follows the '
 "final days of Fry's career as a journalist in Dublin before he was fired for "
 "his coverage of the IRA's seizure of power in the wake of the September 11 "
 "terrorist attacks ..<n>Fry's character is played by Aidan McBride, who also "
 "appeared in the film's predecessor, The Reader .")


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('This site was created by actor Frank Miller .<n>Miller is best known for '
 "playing Batman in the hit television series 'The Batman'<n>Miller has also "
 "starred in the films 'Miami Vice' and 'Saving Mr. Banks'<n>The Batman movie "
 'will be released in the U.S. on July 20th, 2012 at the Grauman Egyptian '
 'Theatre in Los Angeles, California. For more information on the film and to '
 'see all the cast and crew visit batmanmovie.com .')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

('Comedians Seth MacFarlane and Jay Pharaoh join forces to discuss the latest '
 'in stand-up comedy .<n>They talk about their new movie, "Scrabble The '
 'Movie," and what it\'s like to be in Los Angeles, where they\'ve been '
 'filming for the past few months.<n>The duo also discuss their new book, "The '
 'Book Of Revelations," which they hope to release in the near future.<n>Watch '
 "the full interview on this week's episode of The Daily Show, Saturday, April "
 '5, at 2 p.m. ET on Comedy Central .')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('"Jane D\'Oh!" has been picked up for a pilot series by YouTube Red .<n>The '
 'show stars Rawson Marshall Thurber and Jessica Mathers, who play Jane and '
 'Detective Mathers respectively, on a police force in a small-time '
 'crime-ridden town.<n>Thurber on her co-star: "She made me better in every '
 'scene. You know chemistry is a two way street. And some times there are stop '
 'lights and some times that but you just keep going. Because... traffic, '
 "right? (faux reflective) But I'm not gonna let this show go to my head. It's "
 'all about finding my center. Staying grounded. Staying mindful."')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Brain-to-brain interface (see Pons or DRIFT) called a Conn .<n>Joint vision '
 'shared by Jaeger crew members phenomenon vital signs and to translate nerve '
 'signals to piloting input * ghost-drift.<n> virtual space inside the O/S in '
 "which crew telepresences interact inside the subject's brain + The "
 'Interstice. the rift between our universe and the person crew via direct '
 'neural linkup . LOCCENT. short for Local Command Center, comm.')
{'The Guard': {'rouge1': 0.15748031496062992, 'rouge2': 0.032, 'rougeL': 0.12598425196850394, 'rougeLsum': 0.12598425196850394}, 'Batman Year One': {'rouge1': 0.20304568527918782, 'rouge2': 0.010256410256410256, 'rougeL': 0.13197969543147206, 'rougeLsum': 0.13197969543147206}, 'Inherent Vice': {'rouge1': 0.21212121212121213, 'rouge2': 0.061538461538461535, 'rougeL': 0.15151515151515152, 'rougeLsum': 0.15151515151515152}, 'Colombiana': {'rouge1': 0.16296296296296295, 'rouge2': 0.0, 'rougeL': 0.1037037037037037, 'rougeLsum': 0.10370370370370

In [ ]:
pegasus_long_results = combine_rouge_and_summary_results(pegasus_long_rouge_dict, pegasus_long_summary_dict, 'pegasus_long')
pegasus_long_results.head()

,rouge1,rouge2,rougeL,rougeLsum,pegasus_long_summary
The Guard,0.157480,0.032000,0.125984,0.125984,Final Shooting Draft is a feature film by Repr...
Batman Year One,0.203046,0.010256,0.131980,0.131980,This site was created by actor Frank Miller .<...
Inherent Vice,0.212121,0.061538,0.151515,0.151515,This script is the confidential and proprietar...
Colombiana,0.162963,0.000000,0.103704,0.103704,This is the second in a new series from writer...
Barney s Version,0.218750,0.000000,0.125000,0.125000,Barney's Version by Michael Konyves based on n...


In [ ]:
# save results to a csv
pegasus_long_results.to_csv('/content/pegasus_long_results.csv')
# files.download('/content/pegasus_long_results.csv') 

In [ ]:
# average rouge scores
pegasus_long_results.drop(columns=['pegasus_long_summary']).mean(axis=0)

rouge1       0.174883
rouge2       0.011691
rougeL       0.105706
rougeLsum    0.105706
dtype: float64

## Pegasus Large


---


Max 1024 tokens for pretrained *'google/pegasus-large'* model

In [ ]:
pegasus_large_rouge_dict = {}
pegasus_large_summary_dict = {}

for i in range(30):
    pegasus_large_summary = run_pegasus_large_model(subset_train_df.iloc[i]['bert_dialog'])
    pegasus_large_rouge_scores = calculate_rouge_scores(pegasus_large_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    pegasus_large_summary_dict[title] = pegasus_large_summary
    pegasus_large_rouge_dict[title] = pegasus_large_rouge_scores

print(pegasus_large_rouge_dict)
print(pegasus_large_summary_dict)

In [ ]:
pegasus_large_results = combine_rouge_and_summary_results(pegasus_large_rouge_dict, pegasus_large_summary_dict, 'pegasus_large')
pegasus_large_results.head()

In [ ]:
# save results to a csv
pegasus_large_results.to_csv('/content/pegasus_large_results.csv')
# files.download('/content/pegasus_large_results.csv') 

In [ ]:
# average rouge scores
pegasus_large_results.drop(columns=['pegasus_large_summary']).mean(axis=0)

## LongT5


---

Max 16,384 tokens using pretrained *pszemraj/long-t5-tglobal-base-16384-book-summary* model

In [ ]:
longt5_rouge_dict = {}
longt5_summary_dict = {}

for i in range(30):
    longt5_summary = run_longt5_model(subset_train_df.iloc[i]['bert_dialog'])
    longt5_rouge_scores = calculate_rouge_scores(longt5_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    longt5_summary_dict[title] = longt5_summary
    longt5_rouge_dict[title] = longt5_rouge_scores

print(longt5_rouge_dict)
print(longt5_summary_dict)

/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:784: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


("The narrator tells us that this is the final draft of his play. It's been a "
 "long time since we last saw it, and now it's finally time to see it. We get "
 'a brief description of the scene: "What a beautiful Fucking Day" . A guy '
 "named Aidan Mcbride has just arrived from Ireland. He looks like he's just "
 "had a hard time getting out of the city. He's got a drink in his hand, and "
 'he wants to go back home before Mammy tells him what happened.')
('In this short scene, Frank Miller shows up at the auto shop looking for his '
 "son. He's a little confused about where he's going and what he can do there. "
 'When he gets there, he sees that the job orders are in order and he asks '
 'Bruce to come out and help him out since he looks so bad. Bruce tells him '
 "it's all fine and then he goes off to check on some other things.")


Your max_length is set to 256, but you input_length is only 239. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=119)


("Paul Thomas Anderson's reminiscing about how much he used to love Shasta. "
 "Now that he has an office and everything, he decides it's time to take a "
 'walk. He looks in his phone book and thinks about how great it would be if '
 'everyone had this opportunity to get out of bed and have a little chat with '
 'one of his old co-workers. Then he goes over to the office')
('The narrator tells us that the story is about a young boy named Fabio who '
 "wants to become a doctor, a lawyer, and a friend of Don Luis's. It's been a "
 "long time since we last saw him in his comic form, but now he's back on his "
 "own. He asks her what she's doing with her life, and she responds by telling "
 'him it\'s "studying mama" . She also tells him that if he doesn\'t want to '
 'be anything in life, he has to study for it. This is like being a super-hero '
 'or a Doctor. We learn that Emilio is supposed to come see them someday, but '
 "they haven't heard from him yet.")
('In this short scene, Barn

Your max_length is set to 256, but you input_length is only 252. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=126)


...

('In this chapter, the narrator tells us that every day he gets letters from '
 'people who are praying for him. They say things like "I\'m sorry you did it" '
 'or "I just don\'t want to do it anymore." Nobody ever comes back. People ask '
 "him if he's sorry he did it and he says he is. And then there's the mental "
 "gymnastic thing. It's supposed to help us vent our angers.")
("In this short scene, the action shifts to the Russian camp at Eagle's Nest. "
 'Wombat has lost his position and is unable to retrieve the key that was '
 'given to him by Colonel James Faith. The lieutenant asks why they had to '
 'travel so far to get this information. He says they need someone outside of '
 'the system to keep an eye on the man, but it would be impossible for them to '
 'keep him inside the system.')
('In this short scene, Colin Carberry explains that his father is a socialist '
 'who has just returned from the war in Vietnam and is planning to become '
 'blinded. The crowd gathers to hea

Your max_length is set to 256, but you input_length is only 252. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=126)


("In this chapter, Joe tells his son the story of home. It's about a time when "
 'humans came to Africa and five tribes called the Wakandans. They lived '
 'together until a vision came from the god Bast, who led them to the '
 'Heart-Shaped Herb plant, which gave them superhuman strength and speed. The '
 'first Black Panther was the leader of the tribe, but four tribes agreed that '
 "they would live under the king's rule. The Jabari were isolated in the "
 "mountains because they didn't want to be seen.")
('The narrator tells us that the men are responsible for the deaths of three '
 "of the soldiers. It turns out that they're all brothers, and one of them was "
 "born just before giving birth to the king. In other words, there's no way "
 'anyone could have given birth to a twin at this point in the play. So we '
 "guess it's not coincidence that the two boys were twins when they were born.")
('In this short scene, Hoskins and Lucas are loading up the cannons to shoot '
 'down a d

Your max_length is set to 256, but you input_length is only 249. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=124)


('On January 8, 2016, at the Universal Studios, Helen and Dan are gathered to '
 'mourn the death of their friend and neighbor, Daniel Vincyn Cleavesver. They '
 'also remember his wife, who is pretty. Emma invites everyone in the audience '
 'to share some memories of their own childhood friends and family members who '
 'loved and cared for Daniel. As they do so, Emma whispers to Jude that she '
 'would like to hold a party to remember them all.')
('The narrator tells us that every girl is born with a reputation, and that '
 'some girls are "sweet" and dreaming of books, while others are '
 '"hunchesbacks." Some girls fall between houses, live windows, and become '
 "dirty. He says that the best girls are born dirty because they're born "
 'dirty. When he asks who can give him an example of this kind of choice, he '
 "gets a pretty good one: Waiting. If a guy truly likes you but doesn't know "
 "how to understand, then he knows so much about it. What's more, if you don't "
 'stick ar

In [ ]:
longt5_results = combine_rouge_and_summary_results(longt5_rouge_dict, longt5_summary_dict, 'longt5')
longt5_results.head()

,rouge1,rouge2,rougeL,rougeLsum,longt5_summary
The Guard,0.200000,0.000000,0.114286,0.114286,The narrator tells us that this is the final d...
Batman Year One,0.242424,0.000000,0.131313,0.131313,"In this short scene, Frank Miller shows up at ..."
Inherent Vice,0.161290,0.000000,0.096774,0.096774,Paul Thomas Anderson's reminiscing about how m...
Colombiana,0.233333,0.022472,0.133333,0.133333,The narrator tells us that the story is about ...
Barney s Version,0.166667,0.000000,0.121212,0.121212,"In this short scene, Barney is trying to get h..."


In [ ]:
# save results to a csv
longt5_results.to_csv('/content/longt5_results.csv')
# files.download('/content/longt5_results.csv') 

In [ ]:
# average rouge scores
longt5_results.drop(columns=['longt5_summary']).mean(axis=0)

rouge1       0.214650
rouge2       0.014959
rougeL       0.122838
rougeLsum    0.122838
dtype: float64

# Baseline Models (BERT - Text)
All baseline models will be run with 30 sample movie scripts from random seed 266

## Pegasus Short

---


Max 512 tokens for pretrained *'google/pegasus-xsum'* model

In [ ]:
pegasus_short_rouge_dict = {}
pegasus_short_summary_dict = {}

for i in range(30):
    pegasus_short_summary = run_pegasus_short_model(subset_train_df.iloc[i]['bert_text'])
    pegasus_short_rouge_scores = calculate_rouge_scores(pegasus_short_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    pegasus_short_summary_dict[title] = pegasus_short_summary
    pegasus_short_rouge_dict[title] = pegasus_short_rouge_scores

print(pegasus_short_rouge_dict)
print(pegasus_short_summary_dict)

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('A young man is driving his car along a country road in the early hours of '
 'the morning when he sees a police car approaching from the other side of the '
 'road, with a siren on and a flashing blue light on the dashboard.., he turns '
 'around to see the police car and the siren on, but he does not see the car '
 'in front of him, which is a car full of people who have just been killed in '
 'a car crash in the Republic of Ireland.')


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('A man in his late 50s walks into a dark alley in the middle of the night, '
 'his eyes fixed on a dark-blue car, its headlights glaring into the night '
 "sky, its engine running at low power.'' he turns around and walks back into "
 "the alley.' he takes a deep breath and walks out of the alley, his face "
 'buried in his hands, his body contorted in a grimace, his arms stretched out '
 'across his chest, his head bent over his knees.')


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

('The story begins with Dion waiting in the car in the lot of Canter’s for his '
 'friend Seth to pick up his order of chips and dip in a bucket of mustard. '
 'eth pays and is looking up to the pick-up jar, but doesn’t notice Seth’s '
 'character is sexually immature and stands up in the character’s performance '
 'of All Back To Me, with the words “it’s all back to me” appearing on the '
 'screen.')


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('It\'s been called "the most naked selfie in the world" and it\'s all down to '
 "one man's love of running and his grandma's dislike of selfies.... and his "
 "phone... all at the same time... and he's got a girlfriend... and they're "
 'both doing it live on social media... all in the space of a few minutes... '
 'and... well... just a few seconds... and then it all falls apart.')


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('The following is a list of key players in the development of the US '
 "military's first ever artificial intelligence (AI) robot, the Jaeger, which "
 'was built at the University of California, Los Angeles (UCLA) in the early '
 '1990s: The Jaeger was designed to mimic the mind-reading abilities of human '
 'pilots, but was also capable of responding to a wide range of threats, '
 'including those posed by other space-faring entities, as well as threats '
 'posed by humans.')
{'The Guard': {'rouge1': 0.19117647058823528, 'rouge2': 0.04477611940298508, 'rougeL': 0.14705882352941177, 'rougeLsum': 0.14705882352941177}, 'Batman Year One': {'rouge1': 0.1485148514851485, 'rouge2': 0.01, 'rougeL': 0.12871287128712872, 'rougeLsum': 0.12871287128712872}, 'Inherent Vice': {'rouge1': 0.20143884892086333, 'rouge2': 0.0291970802919708, 'rougeL': 0.12949640287769787, 'rougeLsum': 0.12949640287769787}, 'Colombiana': {'rouge1': 0.043478260869565216, 'rouge2': 0.0, 'rougeL': 0.043478260869565216, 'r

In [ ]:
pegasus_short_results = combine_rouge_and_summary_results(pegasus_short_rouge_dict, pegasus_short_summary_dict, 'pegasus_short')
pegasus_short_results.head()

,rouge1,rouge2,rougeL,rougeLsum,pegasus_short_summary
The Guard,0.191176,0.044776,0.147059,0.147059,A young man is driving his car along a country...
Batman Year One,0.148515,0.010000,0.128713,0.128713,A man in his late 50s walks into a dark alley ...
Inherent Vice,0.201439,0.029197,0.129496,0.129496,"Doc’s in the kitchen, writing on a pad, trying..."
Colombiana,0.043478,0.000000,0.043478,0.043478,All photographs courtesy of the author and cou...
Barney s Version,0.169231,0.000000,0.107692,0.107692,Barney and Miriam are sitting in the living ro...


In [ ]:
# save results to a csv
pegasus_short_results.to_csv('/content/pegasus_short_text_results.csv')
# files.download('/content/pegasus_short_text_results.csv') 

In [ ]:
# average rouge scores
pegasus_short_results.drop(columns=['pegasus_short_summary']).mean(axis=0)

rouge1       0.209115
rouge2       0.023188
rougeL       0.130840
rougeLsum    0.130840
dtype: float64

## Pegasus Long


---


Max 1024 tokens for pretrained *'google/pegasus-cnn_dailymail'* model

In [ ]:
pegasus_long_rouge_dict = {}
pegasus_long_summary_dict = {}

for i in range(30):
    pegasus_long_summary = run_pegasus_long_model(subset_train_df.iloc[i]['bert_text'])
    pegasus_long_rouge_scores = calculate_rouge_scores(pegasus_long_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    pegasus_long_summary_dict[title] = pegasus_long_summary
    pegasus_long_rouge_dict[title] = pegasus_long_rouge_scores

print(pegasus_long_rouge_dict)
print(pegasus_long_summary_dict)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Five young men pass a whiskey bottle around -- Public Enemy playing on the '
 'CD player .<n> Garda Sergeant GERRY BOYLE behind the wheel. Staring off into '
 'space.<n>The red car zooms along at breakneck speed -- Eventually shooting '
 'past BoyLE in his police car -- Boyle barely reacts -- Suddenly there comes '
 'the sound of screeching brakes, and the boom of a high-impact car crash -- '
 'boyLE unhurriedly starts the car.')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('A sweating, tormented sleeper  BRUCE WAYNE. His eyes hollowed, skin pale, he '
 'TOSSES and TURNS.<n>Awoken from his NIGHTMARE by the sound of his own '
 'scream, disoriented, BRUCE goes to the sink and SPLASHES cold water in his '
 'face.<n>Little AL, a gigantic, early middle-aged black man carries a bag of '
 'doughnuts and two coffees. He whistles as he walks through a run-down slum.')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

('eth pays and is about to drop a dollar in the tip jar, but notices the '
 'cashier isn’t looking.<n>Jay turns to see Seth, and has a slight “busted” 1 '
 'his face.<n>Seth looks kind of hurt. his eyes. Jay doesn’t notice. a weed '
 'scone.<n>Set! character with bullets. Jay stands up and raises hi in '
 'triumph. Jay starts singing to the tune of in, Dion’s “It’s All Coming Back '
 'To Me” as he perf sexually graphic and immature')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Hilarious video shows Ryan walking up Runyon, taking selfies and shooting '
 'videos as they "hike"<n>Behind Ryan, a VAMPY GIRL in crazy skimpy booty '
 'shorts, holds her phone down below her ass then brings it up to her face '
 'where she smiles and gives a peace sign .<n>Ryan, sporting a very strange '
 "18TH Century ARMY OFFICER'S UNIFORM, lands on the living room floor ..<n>At "
 'the top of Runyon he pans the hazy skyline, landing lying on the Living Room '
 'floor.')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Anteverse. the older, parent universe on the other side of the * '
 'conn-pod.<n>The main campus of the Cosdec Defence College leftover. (slang) '
 "the surviving crewmate of a Jaeger operator Midway.<n>HQ for COSDEC's "
 'ultra-secret Science Division uninhabitable from radiological and biological '
 'contaminants of The Precursors. the unofficial Sci-Division code name for '
 'the hypothetical entities .<n>an ultra- secret research arm of V-50 '
 'Jumphawk.')
{'The Guard': {'rouge1': 0.17094017094017094, 'rouge2': 0.034782608695652174, 'rougeL': 0.10256410256410256, 'rougeLsum': 0.10256410256410256}, 'Batman Year One': {'rouge1': 0.18181818181818182, 'rouge2': 0.02162162162162162, 'rougeL': 0.1283422459893048, 'rougeLsum': 0.1283422459893048}, 'Inherent Vice': {'rouge1': 0.21686746987951808, 'rouge2': 0.024390243902439022, 'rougeL': 0.12048192771084337, 'rougeLsum': 0.12048192771084337}, 'Colombiana': {'rouge1': 0.08633093525179857, 'rouge2': 0.0, 'rougeL': 0.05755395683453237, 'ro

In [ ]:
pegasus_long_results = combine_rouge_and_summary_results(pegasus_long_rouge_dict, pegasus_long_summary_dict, 'pegasus_long')
pegasus_long_results.head()

,rouge1,rouge2,rougeL,rougeLsum,pegasus_long_summary
The Guard,0.170940,0.034783,0.102564,0.102564,Five young men pass a whiskey bottle around --...
Batman Year One,0.181818,0.021622,0.128342,0.128342,"A sweating, tormented sleeper BRUCE WAYNE. Hi..."
Inherent Vice,0.216867,0.024390,0.120482,0.120482,Doc SportELLO is half- awake on his couch. He ...
Colombiana,0.086331,0.000000,0.057554,0.057554,Lara Croft and the Tomb of the Unknowns is pub...
Barney s Version,0.260163,0.033058,0.178862,0.178862,Barney Panofsky's new book is a collection of ...


In [ ]:
# save results to a csv
pegasus_long_results.to_csv('/content/pegasus_long_text_results.csv')
# files.download('/content/pegasus_long_text_results.csv') 

In [ ]:
# average rouge scores
pegasus_long_results.drop(columns=['pegasus_long_summary']).mean(axis=0)

rouge1       0.203990
rouge2       0.017171
rougeL       0.127039
rougeLsum    0.127039
dtype: float64

## Pegasus Large


---


Max 1024 tokens for pretrained *'google/pegasus-large'* model

In [ ]:
pegasus_large_rouge_dict = {}
pegasus_large_summary_dict = {}

for i in range(30):
    pegasus_large_summary = run_pegasus_large_model(subset_train_df.iloc[i]['bert_text'])
    pegasus_large_rouge_scores = calculate_rouge_scores(pegasus_large_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    pegasus_large_summary_dict[title] = pegasus_large_summary
    pegasus_large_rouge_dict[title] = pegasus_large_rouge_scores

print(pegasus_large_rouge_dict)
print(pegasus_large_summary_dict)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('A Garda police car is parked in the road. The red car zooms along at '
 'breakneck speed -- Eventually shooting past BOYLE in his police car -- '
 'BOYLES barely reacts -- Suddenly there comes the sound of screeching brakes. '
 'A YOUNG MAN has been thrown clear, the OTHER FOUR crushed inside the car. '
 'Gets out beside the YOUNG MAN, who is covered in blood. BOYLLE approaches '
 'the concertinaed car, CLOSE on him as he looks inside.')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Awoken from his NIGHTMARE by the sound of his own scream, disoriented, BRUCE '
 'goes to the sink and SPLASHES cold water in his face. LITTLE AL, a gigantic, '
 'early middle-aged black man carries a bag of doughnuts and two coffees. He '
 'stops and inserts a key at a grimy storefront LITTLE AL hits the lights '
 "revealing a meticulously clean LITTLE AL continues his search in the It's a "
 'dreary, dirty, isolated place.')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

('eth pays and is about to drop a dollar in the tip jar, but notices the '
 'cashier isn’t looking. He waits for the cashier to look up, and then drops '
 'his tip in the jar. Jay turns to see Seth, and has a slight “busted” 1 his '
 'face. Jay and Seth are laughing while playing Call of Duty. Jay stands up '
 'and raises hi in triumph. Jay starts singing to the tune of in, Dion’s “It’s '
 'All Coming Back To Me” as he perf sexually graphic and immature')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Ryan pans his camera to show a literal PARADE of SHIRTLESS MEN and '
 'HALF-NAKED WOMEN with insane bodies all walking up Runyon, taking selfies '
 'and shooting videos as they "hike". Behind Ryan, a VAMPY GIRL in crazy '
 'skimpy booty shorts, holds her phone down below her ass then brings it up to '
 'her face where she smiles and gives a peace sign while holding a package of '
 '"Flat Tummy Tea" she\'s hawking.')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('highly classified research facility and surveillance station; HQ for '
 "COSDEC's ultra-secret Science Division uninhabitable from radiological and "
 'biological contaminants of The Precursors. the unofficial Sci-Division code '
 'name for the hypothetical entities on the other side of the Interstice '
 'Science Division/Sci-Division. an ultra- secret research arm of V-50 '
 "Jumphawk. the Jaeger's detachable cockpit module; sometimes while directly "
 'linked via the Pons; triggered by subconscious stimuli, a generally '
 'involuntary but seldom debilitating + drivesuit.')
{'The Guard': {'rouge1': 0.1487603305785124, 'rouge2': 0.016806722689075633, 'rougeL': 0.08264462809917354, 'rougeLsum': 0.08264462809917354}, 'Batman Year One': {'rouge1': 0.19689119170984457, 'rouge2': 0.020942408376963352, 'rougeL': 0.10362694300518135, 'rougeLsum': 0.10362694300518135}, 'Inherent Vice': {'rouge1': 0.21428571428571427, 'rouge2': 0.0, 'rougeL': 0.09999999999999999, 'rougeLsum': 0.09999999999999

In [ ]:
pegasus_large_results = combine_rouge_and_summary_results(pegasus_large_rouge_dict, pegasus_large_summary_dict, 'pegasus_large')
pegasus_large_results.head()

,rouge1,rouge2,rougeL,rougeLsum,pegasus_large_summary
The Guard,0.148760,0.016807,0.082645,0.082645,A Garda police car is parked in the road. The ...
Batman Year One,0.196891,0.020942,0.103627,0.103627,Awoken from his NIGHTMARE by the sound of his ...
Inherent Vice,0.214286,0.000000,0.100000,0.100000,"He looks up, notices someone standing at his d..."
Colombiana,0.192593,0.015038,0.162963,0.162963,ON A BLACK SCREEN CLOSE ON A PAGE OF A LARA CR...
Barney s Version,0.211382,0.033058,0.146341,0.146341,"Barney hangs up, puffs on his cigar. The bottl..."


In [ ]:
# save results to a csv
pegasus_large_results.to_csv('/content/pegasus_large_text_results.csv')
# files.download('/content/pegasus_large_results.csv') 

In [ ]:
# average rouge scores
pegasus_large_results.drop(columns=['pegasus_large_summary']).mean(axis=0)

rouge1       0.201616
rouge2       0.017361
rougeL       0.124374
rougeLsum    0.124374
dtype: float64

## LongT5


---

Max 16,384 tokens using pretrained *pszemraj/long-t5-tglobal-base-16384-book-summary* model

In [ ]:
longt5_text_rouge_dict = {}
longt5_text_summary_dict = {}

for i in range(1):
    longt5_summary = run_longt5_model(subset_train_df.iloc[i]['bert_text'])
    longt5_rouge_scores = calculate_rouge_scores(longt5_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    longt5_text_summary_dict[title] = longt5_summary
    longt5_text_rouge_dict[title] = longt5_rouge_scores

print(longt5_text_rouge_dict)
print(longt5_text_summary_dict)

In [ ]:
longt5_text_rouge_dict = {}
longt5_text_summary_dict = {}

for i in range(30):
    longt5_summary = run_longt5_model(subset_train_df.iloc[i]['bert_text'])
    longt5_rouge_scores = calculate_rouge_scores(longt5_summary, subset_train_df.iloc[i]['plot_outline'])

    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    longt5_text_summary_dict[title] = longt5_summary
    longt5_text_rouge_dict[title] = longt5_rouge_scores

print(longt5_text_rouge_dict)
print(longt5_text_summary_dict)

/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:784: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


('A red car speeds past a barren landscape as the sun rises. Five young men '
 'pass a bottle of whiskey around and watch a police car drive by. Boyle is '
 'behind the wheel, staring out into space. When the red car shoots past his '
 "police car, Boyle doesn't react. The other cars in the road start to crash, "
 'and one man is killed while the other four are crushed inside the vehicle. '
 'Boys drives up beside the dead man, who has been tossed from the car. He '
 'grabs a finger on his throat and dies. After Boyle returns, he closes the '
 'door on the blackened car so that he can look inside.')
('In this short scene, the action shifts to a dark and desolate place. A '
 'menacing "police" is hovering over a sleeping Bruce Wayne. He awakens from '
 'his nightmarish sleep and goes to the washroom to throw cold water on his '
 'face. The sound of an incomprehensible horror fills him with a scream. When '
 'he wakes up, he looks for a clean place to stay. It is a filthy, run-down '
 'b

Your max_length is set to 256, but you input_length is only 242. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=121)


('In this short scene, a third-world bus passes through the streets of Middle '
 'Eastern cities. A woman wearing a "hajib" -- a traditional Arab headdress -- '
 'sits on the floor and stares at the view. Scarlett Durang, an archeologist '
 'from the United States, is watching with a tiny camcorder attached to her '
 'Hajib. He whispers to it not scared. The border between India and the Middle '
 'East is clearly marked.')
('In this short scene, Jesse and Celeste grow to be best friends. Their hands '
 'are crossed in a "C" shape and their lips are kissed passionately. Jesse is '
 "jealous of the couple's college activities and flirts with them at the same "
 'time. The two finally get married on a beautiful wedding cake. They spend '
 'their honeymoon moving into their home and enjoying themselves as they '
 'dance, read, and discuss their past together.')
('A Spotlight reveals that we see ryan standing next to a pile of stuff. He '
 'unzipps a backpack and puts it beside him. People 

Your max_length is set to 256, but you input_length is only 250. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=125)


('The narrator describes a Politician who travels on a jeep. He is wearing '
 'white shirt, pants, and a yellow hat. His supporters ride in front of him '
 'shouting "Yeah, we support you." They cheer him when he doesn\'t give up. '
 'When they do, he tells them to get out of the car. Everyone panics because '
 'there is a tree blocking the road.')
('The narrator shifts gears and muses about the recent political uprisings in '
 'the U.S., which have taken place over the past several months. He notes that '
 'there are still important people in the country who are marching or planning '
 'to march, such as the National Guard and police. A lock on the wall keeps an '
 "alarm going, and when it is time to break it, a man's hand comes into the "
 'frame. It is then that we find ourselves in a kitchen with a bunch of '
 "furniture shrinking away from us. There are pictures of two kids airman's "
 'uniform and a pretty woman.')
('In this short scene, Charles Kingley speaks with an ardor outs

Your max_length is set to 256, but you input_length is only 252. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=126)


('The two women set up a performance space outside the park. Frances plays the '
 'shitty guitar and Sophie sings. When people come by, they stop and shout '
 '"Cutter to Frances" and Sophie runs through the park with her. They make '
 'sure to keep one of their cases so that they can go into the subway '
 'together. After dinner, Frances reads aloud a sentence from Sincerity and '
 'Accurity by Philippe Lionel Trolling. She smokes in two separate windows.')


Your max_length is set to 256, but you input_length is only 244. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=122)


('In this short scene, a meteorite strikes Africa and changes the landscape. '
 'The five tribes of Africa unite and eventually break apart as war breaks '
 "out. A man named N'jobu shoots at a gun and goes over to the border. As the "
 'light from the sky approaches, kids play pick-up basketball in the '
 'background. An African guy named Nejobu sorting through firearms goes over '
 'toward the edge of the earth.')
('The action shifts to a small, speeding van that is being driven by a young '
 'man who has just died. His mother is screaming in the front of the van and '
 'two other boys are slumped on the passenger seats. Outside, there are many '
 'crashes taking place. Most of the cars contain the corpses of recently '
 'deceased men. Some of the drivers are also killed. There are some minor '
 'flaws in the landscape as well.')
("The narrator tells us that this is the first time we've heard of a "
 "twelve-year old girl named Collett. She's making a bracelet out of beads and "
 "li

In [ ]:
longt5_text_results = combine_rouge_and_summary_results(longt5_text_rouge_dict, longt5_text_summary_dict, 'longt5')
longt5_text_results.head()

,rouge1,rouge2,rougeL,rougeLsum,longt5_summary
The Guard,0.198758,0.025157,0.124224,0.124224,A red car speeds past a barren landscape as th...
Batman Year One,0.282828,0.020408,0.141414,0.141414,"In this short scene, the action shifts to a da..."
Inherent Vice,0.188976,0.016000,0.094488,0.094488,A sweet young woman speaks to Doctor Salerio. ...
Colombiana,0.201439,0.000000,0.129496,0.129496,"On a black cover of a comic book, being copied..."
Barney s Version,0.259542,0.031008,0.167939,0.167939,Barney sits down at the table and smokes a cig...


In [ ]:
# save results to a csv
longt5_text_results.to_csv('/content/longt5_text_results.csv')
# files.download('/content/longt5_text_results.csv') 

In [ ]:
# average rouge scores
longt5_text_results.drop(columns=['longt5_summary']).mean(axis=0)

rouge1       0.228796
rouge2       0.017371
rougeL       0.131008
rougeLsum    0.131008
dtype: float64

# Custom Models (BERT - Dialog)

## Two Stage Short Model
This model combines an extractive summary using TextRank, which is then used for abstractive summary using the *pegasus-xsum* model

In [ ]:
rouge_short_dict = {}
summary_short_dict = {}

rouge_short_scores, summaries_short = execute_two_stage_short_model(0, 30, rouge_short_dict, summary_short_dict, subset_train_df)

print(rouge_short_scores)
print(summaries_short)

1701


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('You know nothing about Sergeant Aidan Boyle, you don’t know anything about '
 'him, you didn’t do anything wrong, you weren’t anything to do with anything, '
 'you’ve never done anything like that, you haven’t done anything wrong at '
 'all, you never did anything to hurt anybody, you did nothing to do anything '
 'to harm anyone, you hadn’t ever done anything to anybody, yeah, I’ve heard '
 'nothing about it, I haven’ta heard anything about it.')


1480


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

("Jay-Z's full statement on the fracas that broke out between him and Kanye "
 'West at the MTV Video Music Awards, in which the rapper said he was "not '
 'going back to there" and "didn\'t get it", after the two men clashed on '
 'stage during the performance of his song "Stronger": I don\'t want to go '
 "back to that place, I'm not going to get it, I’m not gonna get it!")
594


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('So, here\'s what Baz had to say about Ugly\'s "boring" Broadway audition on '
 "last night's episode of America's Got Talent: I'm just so bored with this "
 "whole thing, I can't believe it's all over, I don't know what's going to "
 "happen next, but I can tell you this is just not going to work, I've got "
 'nothing else to say, so here it is...')
1415


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("I'm not sure what it is, but I've been trying to get it to communicate with "
 "me for a long time, and it's just not working... I don't know what's wrong "
 "with it, but it doesn't seem to be working at the moment, so I'll just have "
 "to wait and see what happens... I can't tell you what it looks like, because "
 'I can’t hear anything... I’ve never heard anything like that...')
{'The Guard': {'rouge1': 0.07751937984496124, 'rouge2': 0.0, 'rougeL': 0.07751937984496124, 'rougeLsum': 0.07751937984496124}, 'Batman Year One': {'rouge1': 0.16831683168316833, 'rouge2': 0.01, 'rougeL': 0.10891089108910892, 'rougeLsum': 0.10891089108910892}, 'Inherent Vice': {'rouge1': 0.13636363636363638, 'rouge2': 0.0, 'rougeL': 0.10606060606060606, 'rougeLsum': 0.10606060606060606}, 'Colombiana': {'rouge1': 0.163265306122449, 'rouge2': 0.0, 'rougeL': 0.10884353741496598, 'rougeLsum': 0.10884353741496598}, 'Barney s Version': {'rouge1': 0.13533834586466165, 'rouge2': 0.015267175572519083, 'rougeL': 0.0

In [ ]:
two_stage_short_results = combine_rouge_and_summary_results(rouge_short_dict, summary_short_dict, 'two_stage_short')
two_stage_short_results.head()

,rouge1,rouge2,rougeL,rougeLsum,two_stage_short_summary
The Guard,0.077519,0.000000,0.077519,0.077519,"You know nothing about Sergeant Aidan Boyle, y..."
Batman Year One,0.168317,0.010000,0.108911,0.108911,"I don't know how long I'm going to be here, bu..."
Inherent Vice,0.136364,0.000000,0.106061,0.106061,I'm writing to you because I've been looking i...
Colombiana,0.163265,0.000000,0.108844,0.108844,"It's been a long time since I've been here, bu..."
Barney s Version,0.135338,0.015267,0.090226,0.090226,It's been a long time since I've had a chance ...


In [ ]:
# save results to a csv
two_stage_short_results.to_csv('/content/two_stage_short_results.csv')
# files.download('/content/two_stage_short_results.csv') 

In [ ]:
# average rouge scores
two_stage_short_results.drop(columns=['two_stage_short_summary']).mean(axis=0)

rouge1       0.151861
rouge2       0.009955
rougeL       0.094948
rougeLsum    0.094948
dtype: float64

## Two Stage Long Model
This model combines an extractive summary using TextRank, which is then used for abstractive summary using the *pegasus-cnn_dailymail* model

In [ ]:
rouge_long_dict = {}
summary_long_dict = {}

rouge_long_scores, summaries_long = execute_two_stage_long_model(0, 30, rouge_long_dict, summary_long_dict, subset_train_df)

print(rouge_long_scores)
print(summaries_long)

1701


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('You know nothing about me.(getting worked up)<n>Well why don’t you put in '
 'your report that Sergeant Boyle went out of his fucking way to do you boys a '
 'fucking favour and he got really fucking annoyed when you started asking him '
 'stupid fucking questions about a few missing fucking guns, trying to fucking '
 'catch him out as if he’s just some kind of fucking gobshite!<n>Do you think '
 'he might have met someone here who did something bad to him?')
1480


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("I'm going to take a little something back from those bastards.I don't know "
 'how much longer I can go without your help.<n>I could put in a couple days, '
 'help you out down here, if you want a break.You let her get away?<n>I have '
 "this bad feeling that the only way I'll see you again is if I continue my "
 'life of crime.<n>They tried to make it look like burglars but I think it was '
 "Flass and his crew looking for my file.You and me, we're going have.")
1807


...

("I'm not gonnna let yo Shit!I'm gonna go take a piss.We don't have to go in "
 'there!<n>Oh, is this the new gam “The Blame Danny Game?” W Danny don’t play '
 'that.<n>All I know is I’m not going back out there.How the fuck should I '
 "know where they went?<n>We'll kick your ass out!I don’T want to se with you "
 'stupid eally fucking doing this?<n>A frightened Jay ai see shit.Don’t let me '
 'die!!')
594


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('This whole thing was just a walk & talk one-er.Could be your "thing", plus '
 "you know, mouth props.<n>I'm pretty much like the Zero Dark Thirty of "
 'auditioning -- this case Osama would be the audition, not like a real person '
 "-- and get out.<n>Technically maybe, but Baz's was like wa .<n>Cuz nothing "
 "is keeping me from landing this one. I know what it's like to love something "
 "that doesn't love you back .")
1415


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('There was nothing there that could put out a signal like that.Whatever it '
 'is, it has to be triggered on site.<n>Run my math by Sci-Division.I need to '
 'get this damn thing taken out.<n>You think I want to?You’ll feel it every '
 "time you hear about another attack and you won’t know why...Oh, he wasn't "
 "going to let us down without my signal.<n>I've never even heard of back-to- "
 'back simulations without someone Talk to me!')
{'The Guard': {'rouge1': 0.14925373134328357, 'rouge2': 0.0, 'rougeL': 0.08955223880597014, 'rougeLsum': 0.08955223880597014}, 'Batman Year One': {'rouge1': 0.1643835616438356, 'rouge2': 0.0, 'rougeL': 0.0821917808219178, 'rougeLsum': 0.0821917808219178}, 'Inherent Vice': {'rouge1': 0.1804511278195489, 'rouge2': 0.015267175572519083, 'rougeL': 0.10526315789473684, 'rougeLsum': 0.10526315789473684}, 'Colombiana': {'rouge1': 0.2, 'rouge2': 0.0, 'rougeL': 0.12, 'rougeLsum': 0.12}, 'Barney s Version': {'rouge1': 0.05882352941176471, 'rouge2': 0.0, 'rougeL

In [ ]:
two_stage_long_results = combine_rouge_and_summary_results(rouge_long_dict, summary_long_dict, 'two_stage_long')
two_stage_long_results.head()

,rouge1,rouge2,rougeL,rougeLsum,two_stage_long_summary
The Guard,0.149254,0.000000,0.089552,0.089552,You know nothing about me.(getting worked up)<...
Batman Year One,0.164384,0.000000,0.082192,0.082192,I'm going to take a little something back from...
Inherent Vice,0.180451,0.015267,0.105263,0.105263,"Shasta has disappeared, too, just like her boy..."
Colombiana,0.200000,0.000000,0.120000,0.120000,"It's really late, and I'm really really tired ..."
Barney s Version,0.058824,0.000000,0.058824,0.058824,Getting to race in New York was considered hit...


In [ ]:
# save results to a csv
two_stage_long_results.to_csv('/content/two_stage_long_results.csv')
# files.download('/content/two_stage_long_results.csv') 

In [ ]:
# average rouge scores
two_stage_long_results.drop(columns=['two_stage_long_summary']).mean(axis=0)

rouge1       0.150243
rouge2       0.003830
rougeL       0.088418
rougeLsum    0.088418
dtype: float64

## Two Stage Large Model
This model combines an extractive summary using TextRank, which is then used for abstractive summary using the *pegasus-large* model

In [ ]:
rouge_large_dict = {}
summary_large_dict = {}

rouge_large_scores, summaries_large = execute_two_stage_large_model(0, 30, rouge_large_dict, summary_large_dict, subset_train_df)

print(rouge_large_scores)
print(summaries_large)

1701


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('You know nothing about me.(getting worked up) Well why don’t you put in your '
 'report that Sergeant Boyle went out of his fucking way to do you boys a '
 'fucking favour and he got really fucking annoyed when you started asking him '
 'stupid fucking questions about a few missing fucking guns, trying to fucking '
 'catch him out as if he’s just some kind of fucking gobshite!I’m not sure if '
 'you ever get over something like that.I just didn’t realise...')
1480


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("Sooner or later, we'll get lucky and someone will put a bullet in this "
 'nut.Can I get you something?Get out of my way.My 10 (at the door) You know, '
 "maybe there are no real men.I'm going to take a little something back from "
 "those bastards.I don't know how much longer I can go without your help.You "
 'need time to sort stuff out.Maybe if I had kids like the other guys, or had '
 "trouble making ends meet it would be different.He's got to keep moving, like "
 "a shark, You can't stay stuck in one place, Bruce It just ain't HEALTHY.I "
 'think I know a way to find out.You know I COULD put in a couple days, help '
 "you out down here, if you want a break.You let her get away?Looks like it's "
 'going to be a good night, boys.Bring them back whenever you like.I have this '
 "bad feeling that the only way I'm ever going to see you again is if I "
 'continue my life of crime.They tried to make it look like burglars but I '
 'think it was Flass and his crew looking for my file.You

...

("You know... We'll kick your ass out!I don’t want to se with you stupid eally "
 'fucking doing this?That Yeah... A frightened Jay ai see shit.Don’t let me '
 'die!!That’s, like, one of the thing was gonna do while I’m here.He could be '
 'the sociopa letting some psycho i And thanks t Dan ow!Lets go!Let go of '
 "me!ay’ for argument's sake that to fuck up everyone's shit")
594


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('See how we just went in like a big circle but nothing really happened-- That '
 'just did.This whole thing was just a walk & talk one-er.Could be your '
 '"thing", plus you know, mouth props.Make it look like another '
 "murder-suicide?Call me, I can get you work tomorrow.You're like the first "
 "half of A Beautiful Mind.It's like a way to make boring plot stuff less "
 'boring for the audience.(beat) Can I just ask you one thing?Well I guess I '
 'mean it literally.There was no way we were going to let those two steal the '
 'show tonight.If I get it, it could like totally change my career.No, I mean '
 'that in a good way.Cuz nothing is keeping me from landing this one.I know '
 "what it's cool to love something that doesn't love you back.I'm pretty much "
 'like the Zero Dark Thirty of auditioning -- this case Osama would be the '
 'audition, not like a real person -- and get out.Ugly way to go.Im gonna take '
 'a look around back. I have nothing else to say about this.Technicall

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('There was nothing there that could put out a signal like that.Whatever it '
 "is, it has to be triggered on site -- If I'm going to have someone riding a "
 "bomb into the Interstice, I'd like to be certain it will work... Run my math "
 'by Sci-Division.I need to get this damn thing taken out.They were broken '
 "when I got them -- I didn't even think it was possible, but by god, they are "
 'You cannot make them to do this...')
{'The Guard': {'rouge1': 0.14925373134328357, 'rouge2': 0.0, 'rougeL': 0.08955223880597014, 'rougeLsum': 0.08955223880597014}, 'Batman Year One': {'rouge1': 0.19161676646706585, 'rouge2': 0.0, 'rougeL': 0.08982035928143711, 'rougeLsum': 0.08982035928143711}, 'Inherent Vice': {'rouge1': 0.17435897435897435, 'rouge2': 0.010362694300518135, 'rougeL': 0.09230769230769231, 'rougeLsum': 0.09230769230769231}, 'Colombiana': {'rouge1': 0.15748031496062995, 'rouge2': 0.0, 'rougeL': 0.08661417322834646, 'rougeLsum': 0.08661417322834646}, 'Barney s Version': {'rouge1': 

In [ ]:
two_stage_large_results = combine_rouge_and_summary_results(rouge_large_dict, summary_large_dict, 'two_stage_large')
two_stage_large_results.head()

,rouge1,rouge2,rougeL,rougeLsum,two_stage_large_summary
The Guard,0.149254,0.000000,0.089552,0.089552,You know nothing about me.(getting worked up) ...
Batman Year One,0.191617,0.000000,0.089820,0.089820,"Sooner or later, we'll get lucky and someone w..."
Inherent Vice,0.174359,0.010363,0.092308,0.092308,Said Shasta was the only one that ever made an...
Colombiana,0.157480,0.000000,0.086614,0.086614,You know what time it is?We'll get all this st...
Barney s Version,0.104839,0.000000,0.064516,0.064516,"Oh, I know - have you read, uh, what's it call..."


In [ ]:
# save results to a csv
two_stage_large_results.to_csv('/content/two_stage_large_results.csv')
# files.download('/content/two_stage_large_results.csv') 

In [ ]:
# average rouge scores
two_stage_large_results.drop(columns=['two_stage_large_summary']).mean(axis=0)

rouge1       0.157633
rouge2       0.004987
rougeL       0.087577
rougeLsum    0.087577
dtype: float64

## Two Stage Long T5 Model
This model combines an extractive summary using TextRank, which is then used for abstractive summary using the *LongT5* model

In [ ]:
rouge_t5_dict = {}
summary_t5_dict = {}

rouge_t5_scores, summaries_t5 = execute_two_stage_t5_model(0, 30, rouge_t5_dict, summary_t5_dict, subset_train_df)

print(rouge_t5_scores)
print(summaries_t5)

1701


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('You know nothing about me.(getting worked up) Well why don’t you put in your '
 'report that Sergeant Boyle went out of his fucking way to do you boys a '
 'fucking favour and he got really fucking annoyed when you started asking him '
 'stupid fucking questions about a few missing fucking guns, trying to fucking '
 'catch him out as if he’s just some kind of fucking gobshite!I’m not sure if '
 'you ever get over something like that.I just didn’t realise...')


1480


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

("You know... We'll kick your ass out!I don’t want to se with you stupid eally "
 'fucking doing this?That Yeah... A frightened Jay ai see shit.Don’t let me '
 'die!!That’s, like, one of the thing was gonna do while I’m here.He could be '
 'the sociopa letting some psycho i And thanks t Dan ow!Lets go!Let go of '
 "me!ay’ for argument's sake that to fuck up everyone's shit")
594


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('See how we just went in like a big circle but nothing really happened-- That '
 'just did.This whole thing was just a walk & talk one-er.Could be your '
 '"thing", plus you know, mouth props.Make it look like another '
 "murder-suicide?Call me, I can get you work tomorrow.You're like the first "
 "half of A Beautiful Mind.It's like a way to make boring plot stuff less "
 'boring for the audience.(beat) Can I just ask you one thing?Well I guess I '
 'mean it literally.There was no way we were going to let those two steal the '
 'show tonight.If I get it, it could like totally change my career.No, I mean '
 'that in a good way.Cuz nothing is keeping me from landing this one.I know '
 "what it's cool to love something that doesn't love you back.I'm pretty much "
 'like the Zero Dark Thirty of auditioning -- this case Osama would be the '
 'audition, not like a real person -- and get out.Ugly way to go.Im gonna take '
 'a look around back. I have nothing else to say about this.Technicall

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('There was nothing there that could put out a signal like that.Whatever it '
 "is, it has to be triggered on site -- If I'm going to have someone riding a "
 "bomb into the Interstice, I'd like to be certain it will work... Run my math "
 'by Sci-Division.I need to get this damn thing taken out.They were broken '
 "when I got them -- I didn't even think it was possible, but by god, they are "
 'You cannot make them to do this...')
{'The Guard': {'rouge1': 0.14925373134328357, 'rouge2': 0.0, 'rougeL': 0.08955223880597014, 'rougeLsum': 0.08955223880597014}, 'Batman Year One': {'rouge1': 0.19161676646706585, 'rouge2': 0.0, 'rougeL': 0.08982035928143711, 'rougeLsum': 0.08982035928143711}, 'Inherent Vice': {'rouge1': 0.17435897435897435, 'rouge2': 0.010362694300518135, 'rougeL': 0.09230769230769231, 'rougeLsum': 0.09230769230769231}, 'Colombiana': {'rouge1': 0.15748031496062995, 'rouge2': 0.0, 'rougeL': 0.08661417322834646, 'rougeLsum': 0.08661417322834646}, 'Barney s Version': {'rouge1': 

In [ ]:
two_stage_t5_results = combine_rouge_and_summary_results(rouge_t5_dict, summary_t5_dict, 'two_stage_t5')
two_stage_t5_results.head()

,rouge1,rouge2,rougeL,rougeLsum,two_stage_t5_summary
The Guard,0.149254,0.000000,0.089552,0.089552,You know nothing about me.(getting worked up) ...
Batman Year One,0.191617,0.000000,0.089820,0.089820,"Sooner or later, we'll get lucky and someone w..."
Inherent Vice,0.174359,0.010363,0.092308,0.092308,Said Shasta was the only one that ever made an...
Colombiana,0.157480,0.000000,0.086614,0.086614,You know what time it is?We'll get all this st...
Barney s Version,0.104839,0.000000,0.064516,0.064516,"Oh, I know - have you read, uh, what's it call..."


In [ ]:
# save results to a csv
two_stage_t5_results.to_csv('/content/two_stage_t5_results.csv')
# files.download('/content/two_stage_t5_results.csv') 

In [ ]:
# average rouge scores
two_stage_t5_results.drop(columns=['two_stage_t5_summary']).mean(axis=0)

rouge1       0.157633
rouge2       0.004987
rougeL       0.087577
rougeLsum    0.087577
dtype: float64

# Custom Models (BERT - Text)

## Two Stage Short Model
This model combines an extractive summary using TextRank, which is then used for abstractive summary using the *pegasus-xsum* model

In [ ]:
rouge_short_dict = {}
summary_short_dict = {}

rouge_short_scores, summaries_short = execute_two_stage_text_short_model(0, 30, rouge_short_dict, summary_short_dict, subset_train_df)

print(rouge_short_scores)
print(summaries_short)

846


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Three men are sitting in a car, one with a gun in his hand, the other with a '
 'big smile on his face, the third with a blank look in his eye, as if to say, '
 '"I don\'t care, I\'m just going to go home and have a good night\'s sleep, '
 'but I want to know what\'s going to happen to you when I get home."<n>The '
 'three men look at each other with blank eyes.')
1112


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("The cops are on their way, the SWAT team are on the way, and the dude who's "
 "been on the run for a year is finally about to be caught. He's got a gun in "
 "his hand and he's going to take it out on the cops, but they're not going to "
 'let him take it without a fight, so he grabs the gun out of their hands and '
 'shoots them in the head, then takes off running.')
374


...

('Jay and Craig are sitting in the living room with a cup of tea when they are '
 "interrupted by a knock at the door from a man dressed in black who says he's "
 "from the police and wants to talk to them about a case they've been involved "
 'in in the past few days.., they run to the kitchen where they are joined by '
 'Seth, who is dressed in a black suit and white shirt, who runs up to them '
 'with a knife in his hand.')
251


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("It's the final episode of The Only Way Is Essex and it's all about the final "
 'three contestants - Ryan, Parker and Mathers - as they prepare to say their '
 'final goodbyes to the show and move on to bigger and better things in their '
 "lives - and we've got all the details in this week's edition of What's Up "
 'With That Girl?, which airs on Channel 4 on Sunday night at 19:30 BST.')
1717


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("They're sitting on the edge of the bed, arms interlocked, palms facing each "
 'other, eyes fixed on the floor, fingers hovering over the edges of the '
 'frame, fingers pointing at the ceiling, arms hovering above the bed as if '
 "they're waiting for the wind to blow them away, hands hovering over their "
 "eyes as if to warn them of what's about to happen, palms hovering above "
 'their heads as if warning them of the impending disaster..')
{'The Guard': {'rouge1': 0.1705426356589147, 'rouge2': 0.015748031496062992, 'rougeL': 0.10852713178294573, 'rougeLsum': 0.10852713178294573}, 'Batman Year One': {'rouge1': 0.30845771144278605, 'rouge2': 0.020100502512562814, 'rougeL': 0.15920398009950248, 'rougeLsum': 0.15920398009950248}, 'Inherent Vice': {'rouge1': 0.16417910447761197, 'rouge2': 0.0, 'rougeL': 0.1044776119402985, 'rougeLsum': 0.1044776119402985}, 'Colombiana': {'rouge1': 0.23943661971830985, 'rouge2': 0.0, 'rougeL': 0.15492957746478872, 'rougeLsum': 0.15492957746478872}, 'Ba

In [ ]:
two_stage_short_results = combine_rouge_and_summary_results(rouge_short_dict, summary_short_dict, 'two_stage_short')
two_stage_short_results.head()

,rouge1,rouge2,rougeL,rougeLsum,two_stage_short_summary
The Guard,0.170543,0.015748,0.108527,0.108527,"Three men are sitting in a car, one with a gun..."
Batman Year One,0.308458,0.020101,0.159204,0.159204,"The cops are on their way, the SWAT team are o..."
Inherent Vice,0.164179,0.000000,0.104478,0.104478,Bigfoot's visit to Doc's office has been captu...
Colombiana,0.239437,0.000000,0.154930,0.154930,Cat's eyes light up as she looks out of the wi...
Barney s Version,0.194444,0.000000,0.125000,0.125000,Barney and Miriam's last night together before...


In [ ]:
# save results to a csv
two_stage_short_results.to_csv('/content/two_stage_text_short_results.csv')
# files.download('/content/two_stage_text_short_results.csv') 

In [ ]:
# average rouge scores
two_stage_short_results.drop(columns=['two_stage_short_summary']).mean(axis=0)

rouge1       0.218762
rouge2       0.019177
rougeL       0.135875
rougeLsum    0.135875
dtype: float64

## Two Stage Long Model
This model combines an extractive summary using TextRank, which is then used for abstractive summary using the *pegasus-cnn_dailymail* model

In [ ]:
rouge_long_dict = {}
summary_long_dict = {}

rouge_long_scores, summaries_long = execute_two_stage_text_long_model(0, 30, rouge_long_dict, summary_long_dict, subset_train_df)

print(rouge_long_scores)
print(summaries_long)

846


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('BOYLE leans back into the pillows and sighs, a big grin on his face.<n>Two '
 'little girls are looking on. STANTON, who is still seated.They look blankly '
 "at one another.<n>O'LEARY is sitting in an armchair, a gun in one hand, "
 'pointed at BOYle, a whiskey in the other.<n>As CORNELL walks off, he makes a '
 'sinister turn and looks back at STANTON and MOODY, giving them pause.')


1112


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

('Jay opens the bathroom door to find Michael Cera with one girl blowing him '
 'and another eating his ass.His eyes go wide, his jaw drops, and he screams '
 'like a little girl.<n>Seth, Jay and Franco reach the gars getting cornered '
 'by the large, over to the guys and sees“ smiles, then looks back at 1 speed '
 'towards his doom.<n>Jay looks to knife from the kitchen, and throws it laps, '
 'sending the guys CRASHING back into the house.<n>They look up and see Seth '
 "and Jay take off running, as they smash into thing ING BACK TO FRANCO'S!")
251


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Mathers tries to buck him up... love aftermath scenes.clearly replaced the '
 'fake poison in Ryan turns to Priya .<n>Behind Ryan, a VAMPY GIRL in crazy '
 'skimpy booty shorts, holds her phone down below her ass then brings it up to '
 'her face where she smiles and gives a peace sign.<n>Jared, sporting a cast '
 'on his broken arm, shoves Ryan and Mathers DEEPER BACK STAGE, away from '
 'everyone, as Parker joins his side smiling smugly.')
1717


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Flick packs a suitcase, her eyes still red, her hands still shaking '
 'slightly.<n>She hears a deafening ROAR and turns to see another memory '
 'playing out in the water just off the Pier -- Two vast figures clashing in '
 "the surf washes over Flick's face as she watches.<n>Without turning on the "
 'lights she drags one out into the middle of the floor and strains to pry '
 'open the lid with her bare hands -- she doesn’t know what kind of time she '
 'has.')
{'The Guard': {'rouge1': 0.16528925619834708, 'rouge2': 0.016806722689075633, 'rougeL': 0.09917355371900827, 'rougeLsum': 0.09917355371900827}, 'Batman Year One': {'rouge1': 0.16494845360824742, 'rouge2': 0.010416666666666668, 'rougeL': 0.1134020618556701, 'rougeLsum': 0.1134020618556701}, 'Inherent Vice': {'rouge1': 0.10294117647058824, 'rouge2': 0.0, 'rougeL': 0.07352941176470588, 'rougeLsum': 0.07352941176470588}, 'Colombiana': {'rouge1': 0.18055555555555552, 'rouge2': 0.0, 'rougeL': 0.125, 'rougeLsum': 0.125}, 'Barney s 

In [ ]:
two_stage_long_results = combine_rouge_and_summary_results(rouge_long_dict, summary_long_dict, 'two_stage_long')
two_stage_long_results.head()

,rouge1,rouge2,rougeL,rougeLsum,two_stage_long_summary
The Guard,0.165289,0.016807,0.099174,0.099174,"BOYLE leans back into the pillows and sighs, a..."
Batman Year One,0.164948,0.010417,0.113402,0.113402,"While FLASS looks at the cuffs FLASS, angry, t..."
Inherent Vice,0.102941,0.000000,0.073529,0.073529,Bigfoot leads Doc down a hall and into a room ...
Colombiana,0.180556,0.000000,0.125000,0.125000,Cat and the MONITOR wake up is at seven.<n>Cat...
Barney s Version,0.193103,0.000000,0.110345,0.110345,Barney and IZZY dance on the dancefloor .<n>Mi...


In [ ]:
# save results to a csv
two_stage_long_results.to_csv('/content/two_stage_text_long_results.csv')
# files.download('/content/two_stage_text_long_results.csv') 

In [ ]:
# average rouge scores
two_stage_long_results.drop(columns=['two_stage_long_summary']).mean(axis=0)

rouge1       0.207879
rouge2       0.012080
rougeL       0.116712
rougeLsum    0.116712
dtype: float64

## Two Stage Large Model
This model combines an extractive summary using TextRank, which is then used for abstractive summary using the *pegasus-large* model

In [ ]:
rouge_large_dict = {}
summary_large_dict = {}

rouge_large_scores, summaries_large = execute_two_stage_text_large_model(0, 30, rouge_large_dict, summary_large_dict, subset_train_df)

print(rouge_large_scores)
print(summaries_large)

846


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('OVERHEAD SHOT -- BOYLE leans back into the pillows and sighs, a big grin on '
 'his face.Then looks back at BOYLLE.Gets out, looks around -- TWO LITTLE '
 'GIRLS are looking on.EVERETT watches her go, then looks at STANTON, who is '
 'still seated.They look blankly at one another.They are blankly up at ONE '
 'another.Then at the BOYLES, who are standing nearby, playing with the big '
 "toe of another corpse, a blank look on their face.He turns around -- O'LEARY "
 'is sitting in an armchair, a gun in one hand, pointed at BOYLE, a whiskey in '
 'the other.BOYLE enters and looks around, grinning -- A big map.EVEREET looks '
 'on, impressed, as the MAN turns towards the strand and swims in.As CORNELL '
 'walks off, he makes a sinister turn and looks backat STANTON and MOODY, '
 'giving them pause.He and SHEEHY look at one other, until SHEEhy becomes '
 'aware of the song that is playing --  BOY LE shoots a glance at AOIFE.Then '
 'he hears a sound and looks up -- ’BOYLE’s')
1112


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('While FLASS looks at the cuffs, FLASS, angry, throws them back and SLAMS the '
 'CLOSET DOOR  Just at the moment BRUCE CRASHES into the rat-infested GORDON '
 'hearing something, takes a look out the window where BRUCE was hanging '
 'moments ago.Upstairs, the SWAT TEAM, guns poised, peeks over the lip of the '
 'broken ceiling just in time to get a face full of  Outside the BATHROOM '
 'door, THE BAT-MAN covers his ears until the debris settles.But he COMES UP '
 'SHOOTING DOUBLE HEAD SHOT  blood sprays, one down  TWO CHEST SHOTS  a second '
 'cop staggers back, stricken and ONE IN THE HIP  third cop spins, but manages '
 'to keep his feet, until  BLAM!He hangs four stories up by his fingers as '
 'inside  Then, confronted with the sight of his DEAD PARTNER  Out the window '
 'BRUCE is slipping.Once she is close enough to the guards, she whips off her  '
 'the first one across the face with the buckle, knocking his  her long nails '
 "across the second's face.After a moment BAVEN st

...

('Jay opens the bathroom door to find Michael Cera with one girl blowing him '
 'and another eating his ass.His eyes go wide, his jaw drops, and he screams '
 'like a little girl.Seth, Jay and Franco reach the gars getting cornered by '
 'the large, over to the guys and sees“ smiles, then looks back at 1 speed '
 'towards his doom.Jay looks to knife from the kitchen, and throws it laps, '
 'sending the guys CRASHING back into the house.They look up and see Seth and '
 "Jay take off running, as they smash into thing ING BACK TO FRANCO'S!Jay "
 'flicks the lighter again, and this time, the FLAME LIGHTS and we see WHAT '
 'APPEARS TO BE A MONSTER standing right beside them.The guys slowly WALK into '
 'the living room and wriggling around on the floor, his eyes roli Jay and '
 'Craig tip toe through the house, their weapons held high.They looked away '
 'away from trying the same thing.Thick black s Jayand Craig hold hands as '
 'they slowly make their way through the smoke.Jonah slams in

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('light smile lines coming in here and Mathers and Ryan talk with a '
 'scarf-wearing ALFONSO DIAPHANO (60s) who flutters about the space getting '
 'things ready for the big show tonight.Behind Ryan, a VAMPY GIRL in crazy '
 'skimpy booty shorts, holds her phone down below her ass then brings it up to '
 'her face where she smiles and gives a peace sign while holding a package of '
 '"Flat Tummy Tea" she\'s hawking.Reveal Jared, sporting a cast on his broken '
 "arm, shoves Ryan and Mather's DEEPER BACK STAGE, away from everyone, as "
 'Parker joins his side smiling smugly.Then we hear the cry... Parker spins '
 'just in time to take a crisp right cross to the jaw from Mathers -- '
 'CLUNK.Suddenly, Ryan starts jerk his body all over the')
1717


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('She hears a deafening ROAR and turns to see another memory playing out in '
 'the water just off the Pier -- Two vast figures clashing in the surf washes '
 "over Flick's face as she watches.The shattered windows come back together as "
 'the shock wave lifts Young Mako off the floor, and the roar retreats, going '
 "back into Trespasser's Dread.Without turning on the lights she drags one out "
 'into the middle of the floor and strains to pry open the lid with her bare '
 'hands -- she doesn’t know what kind of time she has.Flick packs a suitcase, '
 'her eyes still red, her hands still shaking slightly.BACK TO PRESENT The '
 'NOISE of the memory echoes as Raleigh pulls himself up to sit on the edge of '
 'the bed, his face in his hands.They both chuckle awkwardly as they stare at '
 'one another across the hall, a thought on the tip of their tongues, '
 "Simultaneously, they realize there's no point in trying to phrase it, and "
 'they let themselves collide in the middle hallway, 

In [ ]:
two_stage_large_results = combine_rouge_and_summary_results(rouge_large_dict, summary_large_dict, 'two_stage_large')
two_stage_large_results.head()

,rouge1,rouge2,rougeL,rougeLsum,two_stage_large_summary
The Guard,0.133929,0.009009,0.098214,0.098214,OVERHEAD SHOT -- BOYLE leans back into the pil...
Batman Year One,0.200669,0.013468,0.113712,0.113712,"While FLASS looks at the cuffs, FLASS, angry, ..."
Inherent Vice,0.139535,0.000000,0.124031,0.124031,DOC gives him a PHOTO OF COY like The Last Sup...
Colombiana,0.170732,0.000000,0.105691,0.105691,"She flattens against the wall, holding onto th..."
Barney s Version,0.141304,0.010989,0.097826,0.097826,"Off Barney, completely shell shocked we -- PRE..."


In [ ]:
# save results to a csv
two_stage_large_results.to_csv('/content/two_stage_text_large_results.csv')
# files.download('/content/two_stage_large_results.csv') 

In [ ]:
# average rouge scores
two_stage_large_results.drop(columns=['two_stage_large_summary']).mean(axis=0)

rouge1       0.202298
rouge2       0.012198
rougeL       0.120102
rougeLsum    0.120102
dtype: float64

The code below exists because the above model timed out and did not save to a .csv

In [ ]:
rouge_large_dict = {'The Guard': {'rouge1': 0.13392857142857142, 'rouge2': 0.009009009009009009, 'rougeL': 0.09821428571428573, 'rougeLsum': 0.09821428571428573}, 'Batman Year One': {'rouge1': 0.20066889632107024, 'rouge2': 0.01346801346801347, 'rougeL': 0.11371237458193979, 'rougeLsum': 0.11371237458193979}, 'Inherent Vice': {'rouge1': 0.13953488372093026, 'rouge2': 0.0, 'rougeL': 0.12403100775193798, 'rougeLsum': 0.12403100775193798}, 'Colombiana': {'rouge1': 0.17073170731707316, 'rouge2': 0.0, 'rougeL': 0.1056910569105691, 'rougeLsum': 0.1056910569105691}, 'Barney s Version': {'rouge1': 0.14130434782608697, 'rouge2': 0.010989010989010988, 'rougeL': 0.09782608695652174, 'rougeLsum': 0.09782608695652174}, 'Zootopia': {'rouge1': 0.22754491017964074, 'rouge2': 0.012121212121212121, 'rougeL': 0.1437125748502994, 'rougeLsum': 0.1437125748502994}, 'Scary Stories to Tell in the Dark': {'rouge1': 0.21686746987951805, 'rouge2': 0.032388663967611336, 'rougeL': 0.12048192771084337, 'rougeLsum': 0.12048192771084337}, 'The Edge of Seventeen': {'rouge1': 0.15624999999999997, 'rouge2': 0.0, 'rougeL': 0.09374999999999999, 'rougeLsum': 0.09374999999999999}, 'As Above So Below': {'rouge1': 0.17843866171003717, 'rouge2': 0.022471910112359546, 'rougeL': 0.13382899628252787, 'rougeLsum': 0.13382899628252787}, 'Celeste Jesse Forever': {'rouge1': 0.22053231939163498, 'rouge2': 0.030651340996168584, 'rougeL': 0.12927756653992395, 'rougeLsum': 0.12927756653992395}, 'Up in the Air': {'rouge1': 0.14062500000000003, 'rouge2': 0.0, 'rougeL': 0.078125, 'rougeLsum': 0.078125}, 'Newton': {'rouge1': 0.28484848484848485, 'rouge2': 0.03048780487804878, 'rougeL': 0.1333333333333333, 'rougeLsum': 0.1333333333333333}, 'Mark Felt The Man Who Brought Down the White House': {'rouge1': 0.24647887323943662, 'rouge2': 0.021201413427561842, 'rougeL': 0.11267605633802817, 'rougeLsum': 0.11267605633802817}, 'Alice in Wonderland': {'rouge1': 0.2760942760942761, 'rouge2': 0.006779661016949153, 'rougeL': 0.1548821548821549, 'rougeLsum': 0.1548821548821549}, 'Rise of the Planet of the Apes': {'rouge1': 0.227027027027027, 'rouge2': 0.010928961748633882, 'rougeL': 0.12972972972972974, 'rougeLsum': 0.12972972972972974}, 'Jennifer s Body': {'rouge1': 0.1698841698841699, 'rouge2': 0.0, 'rougeL': 0.09266409266409266, 'rougeLsum': 0.09266409266409266}, 'MacGruber': {'rouge1': 0.22641509433962265, 'rouge2': 0.0076045627376425855, 'rougeL': 0.13584905660377358, 'rougeLsum': 0.13584905660377358}, 'Good Vibrations': {'rouge1': 0.1755485893416928, 'rouge2': 0.006309148264984227, 'rougeL': 0.10031347962382445, 'rougeLsum': 0.10031347962382445}, 'Frances Ha': {'rouge1': 0.21052631578947367, 'rouge2': 0.022727272727272728, 'rougeL': 0.12030075187969924, 'rougeLsum': 0.12030075187969924}, 'Black Panther': {'rouge1': 0.1951219512195122, 'rouge2': 0.012345679012345678, 'rougeL': 0.10975609756097561, 'rougeLsum': 0.10975609756097561}, 'Y The Last Man': {'rouge1': 0.13675213675213677, 'rouge2': 0.008620689655172414, 'rougeL': 0.10256410256410257, 'rougeLsum': 0.10256410256410257}, 'Shadow Dancer': {'rouge1': 0.19047619047619047, 'rouge2': 0.02214022140221402, 'rougeL': 0.14652014652014653, 'rougeLsum': 0.14652014652014653}, 'Secretariat': {'rouge1': 0.16149068322981366, 'rouge2': 0.0, 'rougeL': 0.11180124223602485, 'rougeLsum': 0.11180124223602485}, 'Dunkirk': {'rouge1': 0.20853080568720378, 'rouge2': 0.009569377990430622, 'rougeL': 0.14218009478672985, 'rougeLsum': 0.14218009478672985}, 'Hold the Dark': {'rouge1': 0.2406015037593985, 'rouge2': 0.01515151515151515, 'rougeL': 0.11278195488721805, 'rougeLsum': 0.11278195488721805}, 'Bridget Jones s Baby': {'rouge1': 0.19455252918287935, 'rouge2': 0.00784313725490196, 'rougeL': 0.10116731517509728, 'rougeLsum': 0.10116731517509728}, 'Dirty Girl': {'rouge1': 0.27672955974842767, 'rouge2': 0.012738853503184712, 'rougeL': 0.13836477987421383, 'rougeLsum': 0.13836477987421383}, 'This Is the End': {'rouge1': 0.254416961130742, 'rouge2': 0.021352313167259784, 'rougeL': 0.1625441696113074, 'rougeLsum': 0.1625441696113074}, 'Ryan Hansen Solves Crimes on Television': {'rouge1': 0.2272727272727273, 'rouge2': 0.0, 'rougeL': 0.11818181818181818, 'rougeLsum': 0.11818181818181818}, 'Pacific Rim': {'rouge1': 0.23974763406940064, 'rouge2': 0.019047619047619053, 'rougeL': 0.13880126182965302, 'rougeLsum': 0.13880126182965302}}
rouge_large_dict

{'The Guard': {'rouge1': 0.13392857142857142,
  'rouge2': 0.009009009009009009,
  'rougeL': 0.09821428571428573,
  'rougeLsum': 0.09821428571428573},
 'Batman Year One': {'rouge1': 0.20066889632107024,
  'rouge2': 0.01346801346801347,
  'rougeL': 0.11371237458193979,
  'rougeLsum': 0.11371237458193979},
 'Inherent Vice': {'rouge1': 0.13953488372093026,
  'rouge2': 0.0,
  'rougeL': 0.12403100775193798,
  'rougeLsum': 0.12403100775193798},
 'Colombiana': {'rouge1': 0.17073170731707316,
  'rouge2': 0.0,
  'rougeL': 0.1056910569105691,
  'rougeLsum': 0.1056910569105691},
 'Barney s Version': {'rouge1': 0.14130434782608697,
  'rouge2': 0.010989010989010988,
  'rougeL': 0.09782608695652174,
  'rougeLsum': 0.09782608695652174},
 'Zootopia': {'rouge1': 0.22754491017964074,
  'rouge2': 0.012121212121212121,
  'rougeL': 0.1437125748502994,
  'rougeLsum': 0.1437125748502994},
 'Scary Stories to Tell in the Dark': {'rouge1': 0.21686746987951805,
  'rouge2': 0.032388663967611336,
  'rougeL': 0.1204

In [ ]:
summary_large_dict = {'The Guard': "OVERHEAD SHOT -- BOYLE leans back into the pillows and sighs, a big grin on his face.Then looks back at BOYLLE.Gets out, looks around -- TWO LITTLE GIRLS are looking on.EVERETT watches her go, then looks at STANTON, who is still seated.They look blankly at one another.They are blankly up at ONE another.Then at the BOYLES, who are standing nearby, playing with the big toe of another corpse, a blank look on their face.He turns around -- O'LEARY is sitting in an armchair, a gun in one hand, pointed at BOYLE, a whiskey in the other.BOYLE enters and looks around, grinning -- A big map.EVEREET looks on, impressed, as the MAN turns towards the strand and swims in.As CORNELL walks off, he makes a sinister turn and looks backat STANTON and MOODY, giving them pause.He and SHEEHY look at one other, until SHEEhy becomes aware of the song that is playing --  BOY LE shoots a glance at AOIFE.Then he hears a sound and looks up -- ’BOYLE’s", 'Batman Year One': "While FLASS looks at the cuffs, FLASS, angry, throws them back and SLAMS the CLOSET DOOR  Just at the moment BRUCE CRASHES into the rat-infested GORDON hearing something, takes a look out the window where BRUCE was hanging moments ago.Upstairs, the SWAT TEAM, guns poised, peeks over the lip of the broken ceiling just in time to get a face full of  Outside the BATHROOM door, THE BAT-MAN covers his ears until the debris settles.But he COMES UP SHOOTING DOUBLE HEAD SHOT  blood sprays, one down  TWO CHEST SHOTS  a second cop staggers back, stricken and ONE IN THE HIP  third cop spins, but manages to keep his feet, until  BLAM!He hangs four stories up by his fingers as inside  Then, confronted with the sight of his DEAD PARTNER  Out the window BRUCE is slipping.Once she is close enough to the guards, she whips off her  the first one across the face with the buckle, knocking his  her long nails across the second's face.After a moment BAVEN stands up and they bo", 'Inherent Vice': "DOC gives him a PHOTO OF COY like The Last Supper, Coy as Jesus grabbing food from BIGFOOT looks at Doc, then the picture He motions down a back corridor... Bigfoot leads Doc down a hall and into a room with a Bigfoot it opens up He puts the FROZEN BANANAS in the PNEUMATIC TUBE DISPENSER, sending chocolate frozen bananas hurling Bigfoot's FACE TURNS AND THE MOLECULES COMPLETELY CHANGE AS HE SEES SOMETHING OVER DOC'S SHOULDER...", 'Colombiana': "She flattens against the wall, holding onto the ON THE BATHROOM One of the gunman comes to the window, cannot fit through, looks out, does not see anybody across, then hears something beneath him, looks down, sees the tip of a shoe below.The gate to the garden begins to open, and like a shot the dogs are up, hurling themselves toward the opening gate, eyes flashing, barking loud enough to scare the dead back to life, fangs bared, ready to kill.Some do homework on their beds, others do their hair, dance, And suddenly, at the sound of a door opening, they all stop and look in the same direction at Cat and the MONITOR Wake up is at seven.Cat hesitates for a moment, and then removes one hand from underneath the table, her fist closed tight.A small smile breaks across his face as he looks in his rear view mirror and sees Cat running and firing, losing ground.Let's him kiss her lips, her neck, and as he goes south, kissing as she goes, the CAMERA holds on her face, and for the", 'Barney s Version': "Off Barney, completely shell shocked we -- PRE-LAP - A KNOCKING at the door The door opens revealing KATE, her eyes filled with tears she throws her arms around Barney and won't let go.Barney looks around the big, empty house, going out of his mind, not knowing what to do with himself.Even more gorgeous than the first time he laid eyes on her Barney drops into his seat across from Miriam, startling her Barney doesn't look so good.The BAND playing an upbeat number for a jam-packed dancefloor Barney slips out from the crowd and races to the bar -- CU - A BLACK & WHITE TV hidden behind the bar showing the Barney downs another shot, then looks back to the dancefloor where he sees IZZY slow dancing with The Second Mrs.", 'Zootopia': 'Hopps wipes snow off a back bumper They open the big door, which looks like it belongs on a refrigerator.As Hopps rolls her eyes and goes back to collecting clues, Nick lowers the back partition and his eyes go wide.Nick ushers Hopps to the door, but opens it to find two big The polar bears grab Nick and Hopps by their throats... ...and yank them off screen.She looks around to see animals coming back together.Suddenly, a BIG KID turns off the lights.Polar bear fur... Rat Pack music...', 'Scary Stories to Tell in the Dark': 'and spot the parked CORVAIR MONZA.KLAXONS sound off-- IN FIRST FLOOR HALLWAY Stella and Ramén witness RED WARNING LIGHTS, causing every Ramén skims ‘The Dream’ in the Bellows Book, quickly skipping to the end.A five story fall onto a sidewalk if they don’t make the jump.As Ramén takes in the dreary room, Ramén sees a broken rocking chair next to cleft toys strewn on the ground, and with her words, they take on new meaning.A CHILD who Young Sarah is reading to turns his head to look straight at him, breaking the fourth wall.But then he sees something horrific -- The front door to the house kicks open to a wind-ravaged SHERIFF MILNER!Inside, they share a hopeful glance, but the floor numbers With Sarah just twenty feet behind him, Auggie turns another corner and finds THE ELEVATOR!Ramén double chec', 'The Edge of Seventeen': "She leaves and Nadine drops the smile, then moves to her window, watching the two of them walk down the driveway hand in hand, Darian sweetly helping Krista into the passenger Something about it makes her feel like absolute shit.Just then, Nadine comes up, squeezes herself between Krista and Darian, puts an arm around each, big smile.Krista is folding laundry on her bed when she hears something It almost sounds like Radiohead's Karma Police, except She moves to the open window, looks down to see she's being A PERSIAN MAN (45) -- big mustache, long pony tail, iridescent shirt unbuttoned to reveal copious chest hair and He's standing below her window with a guitar, singing Karma Police in a very thick accent, about 10x more up-tempo, and with lots of little middle-eastern vocal trills,Krista pulls away, They just look at each other for a second, freaked out.We go to an ultra-wide shot of Nadine as a tiny little dot, the only person eating by herself in She immediately grabs", 'As Above So Below': 'George touches the back of his hand to his mouth and sees the Papillon makes a crazy sign with his hands.George fights like mad to keep the Raver at bay, and even gets back to his feet as ANOTHER RAVER RIPS A HUGE BITE OUT OF GEORGE’S NECK, shooting blood into the air.One of them slowly turns his head, LOOKS RIGHT AT ZED AND GEORGE.He shines a light into Underneath the bones is a gutter full of water, in which a YOUNG MAN, struggles to get to the surface of the water to The Young Man pushes against the bones, but George’s weight on top of them makes it impossible for the Young Man to TheYoung Man seems to keep pace with George as he desperately tries to get out of the way.One turns his Head to look at them.Not one to take criticism She turns and goes back to the fire with the others.George is first to get his hands on a flashlight which he flips on and looks for the source of the music.The camera lies right behind Zed and George just catching a piece of the passing Figure', 'Celeste Jesse Forever': 'But her face is still wet with Jesse gives Celeste a big, long hug.Celeste and Jesse giggle at their stupid inside joke for a little too long.Lucky has Celeste in a bear hug and it looks like she may have broken her back.Jesse is mid-laugh and looks at his phone to see Celeste is calling.Bon Iver\'s "Skinny Love" plays as Celeste walks slowly amongst the celebrity impersonators, tourists and drunks on It is intercut with a montage of Super 8 footage of Celeste and Jesse in the past reluctantly cutting a head off a fish to cook it, Celeste is repulsed but laughing.Celiste and Paul walk down stairs into an incredibly cool- looking speak easy.CELESTE\'S LAWYER, male, 40, speaks Celeste looks up, makes eye contact with Jesse and smiles Celeste is still signing.Celesteste can\'t stop herself from constantly looking over at the studio to see if Yogurt Girl is still in there.A beat later, Celeste tears out of the bar, walking quickly after Jesse.Ceste gives Beth\'s mom, CAROL, 60, ver', 'Up in the Air': "almost as if looking through binoculars.Ryan turns, thinks of something, then turns back.RYAN MOVING THROUGH THE SECURITY SCREENING It's a beautiful choreographed ballet of a bag handle collapsing, shoes coming off, a laptop going in a separate tray, wallet and watch sliding into a shoe, a boarding card sliding into back pocket... both hands always moving, Ryan enters and presents his ADMIRALS CLUB CARD.Natalie accidentally makes eye contact with one guy, then quickly shifts her gaze forward.Ryan looks around for a", 'Newton': "Atma Singh looks like a miffed school boy sitting in one Another beep is heard.The old head man who was in the queue, walks towards Priya and shows her his ID.Atma Singh sits on a ledge and listens The two children become quiet upon hearing Newton's voice.Priya is looking for something in the bushes outside the She breaks a stem and comes out, holding it.Newton looks at his father with anger as if asking Two cycle rickshaws- on one, sits Newton, and on the other, Whenever Newton''s rickshaw goes slightly ahead, his father tells his rider to buck up.Another constable comes in front of them and ‘shoots’ him with his finger (shaped like a gun).Atmam Singh takes out his cell phone and directs the camera side towards the boy and clicks a picture.Newton enters the booth to see sits deep in thought with a worried expression on his face.As they walk back, Krishna and Lakhma try to reason with Newton in their own way.He goes and holds the control unit, which has a button which needs to", 'Mark Felt The Man Who Brought Down the White House': 'He stops her, “I want to see you.” She opens to him, brings him inside.At the end of something.Smith waves her to keep going.We hear only Eyes make a subtle shift, no more than the dilation of pupils.She turns him slow, sensual, and we think maybe with love and Now he turns to look at her.come out and unwrap everything.Moving with a kind of imperious grace.Without going to him.Turns right into it.A day like any other.And goes to the sink.And by the sheer force of his love Joan relents and collapses into him, just like that.We HOLD, stay with him.Flattens the list on WE MOVE IN ON THE LIST Felt picks up the phone.Then they move from the folder to Gray’s eyes and HOLD.Bates starts to leave.Gray exits, joining the flow of federal employees on lunch A REFLECTION IN A WINDOW.And without a word or permission he just wraps her and the baby in his arms, justlike that.with force and direction.’ And makes some stabbing motion with his hand.Then he hangs Felt slows up to and stops ou', 'Alice in Wonderland': 'Once inside, they smile, gasp and flatter.She carries it out of the stall, glancing back at the Bandersnatch before going out.He staggers back, gasping.His voice goes hoarse with emotion as he begins the tale.She smiles and goes off.He sits, amused if nothing else.Frustrated, Alice looks out the window and mutters.She wipes a telltale bit of jam from the side of its mouth.She laughs at herself and goes back to the table.She hears a loud WHACK, a small cry of pain, then cheers and laughter.At the end, she whirls to leer into the face of one terrified frog.Lady Long-Ears rushes in and whispers into the Queen’s ear.He leans in for a kiss just as Lady Long Ears passes.something with wicked claws.His face starts to shimmer as he whispers in her ear.She jerks and drags himself back from the edge of hysteria.They laugh.She opens the little door and bends down to look through to the other side.Wind knocked out, she gasps for air, then she sits up, rubbing the bump on her forehead.Long Ear', 'Rise of the Planet of the Apes': "As Will pulls away Caesar strains for a last glimpse, twisting around to look out the back window.AND SOMETHING ELSE We look through cage bars at Will and Franklin, who stand far away, across the room.At the height of his backward motion he catches another Caesar pumps his legs, eager to get a better look... Will's making sandwiches, his eye on Caesar through the WE FIND WALTER just sitting down on the piano bench.HE LOOKS UP TO SEE CAESAR...", 'Jennifer s Body': 'Needy reaches up and distractedly pulls back the neckline In the moonlight, we see Needy still has a infected- looking BITEMARK near her shoulder.Needy rolls her eyes, turns away, and continues limping into the gym.The bile has hundred of tiny SPINES sticking out of it, like little porcupine needles, and it appears to move by itself, oozing across the linoleum, creeping into the Needy grabs Jennifer blindly and claps a hand over her Jennifer falls to her knees, her screams turning to jagged, insane laughter. Needy scrapes the sandwich out of the skillet and takes a She tuns on the kitchen radio and is surprised to hear a Suddenly, she pauses, a look of UNEASE and even FEAR crossing her face.Needly glances around the room and sees students fighting back tears.As she stares out the window, she winds colored yarn around a pair of Popsicle sticks to create a Out a single window, we See an imposing nine-foot Next to Needy, we saw a pile of unopened mail scattered casually on t', 'MacGruber': 'The noises get GUARD POV - looking left and right around the deserted rail yard, finally the camera pans up to find MacGruber on the roof of a small building, dancing around completely naked (except for his boots) with a carrot in his ass.Piper shuts the door then turns back to see MacGrueber with his pants off, braced for a fucking against the desk.MacGruber turns around and starts MacGrubers turns back around, gets down on one knee and presents to Vicki a hastily assembled copper wire ring with a ball bearing diamond.Moments later, MacGrubert runs back in, grabs his shorn hair fragments and takes off again.Macgruber twirls Piper around like a fancy dance move, clutching him against his chest.Vicki’s eyes dart around the room for a way out.Piper and MacGruBER shoot like crazy as they race for the door.Mac Gruber searches his pockets, but instead of a ticket, presents to the man his hand, middle finger sticking out.From above the grave, we see two hands grip the side of the', 'Good Vibrations': 'He hugs each one in turn, unaware quite how little the boys DAVE arrives at TERRI’s shoulder just in time to hear... RUDI look at one another and laugh.TERRI at moments in this scene is picking things up off the floor, the counter RUDI and DAVE look like they agree.TERRELIC plunges his hand in, shakes the eye dry and pops it back in just as BRIAN and RONNIE of RUDI approach.TER RI and RUC MAN finally look at the stage where RUDI are ripping into ‘Cops’.The solo continues On one side of the glass RUDI sit exhausted, but elated.DAVE, RUTH, MARILYN, TERRI sit in a line at the bar watching a tiny black and white TV on which the UNDERTONES play ‘Get Over You’, wearing their usual skinner jeans.TerRI and the CARTOON PUNKS are capering around the living room while Rudi play on TV and TERRI hollers along.TERPRI stands behind DAVY SMYTH at the desk, watching RUDI.DIVE and the other RUDI members are in He runs out of fingers They leave.The others laugh, pat his back.The music is distorted', 'Frances Ha': "Frances gets up does something quickly and sits back Frances has the stove going with a couple of items.(Found Frances comes down an escalator, sees her parents waiting for her, holding a dog.How NOT THE ONE IN THE LIVING ROOM THE Frances is in a T-Shirts that says “ASK ME!” Another girl has on the same T-Shirt.A very annoyed looking girl is waiting for him there Frances tiptoes around the other girl and into her room.On the walk back to her room, she sees a Younger Girl crying.Sophie falls asleep first and Frances quietly puts the computer away.She is really, really dancing her Frances is one of the last ones to get her stuff out.of the bed and put one foot on the Frances nods.Frances comes over and sits next to him, puts her hand on his back.FranceS makes a move out of the bed.Then decides the girl needs someone Frances sits and the Younger Girl cries.He finds the photo he was looking for and hands the phone to Frances across the table.France's glances at the hand, and in", 'Black Panther': "Klaue smiles, dropping his gun and puts his hands up -- as KLAUE'S PROSTHETIC LEFT ARMsuddenly splits open, revealing T'Challa snatches up a ROLLING CHANGECART TABLE just as Klaue fires the blaster at him .Zuri turns back and T'CChalla lifts his hand, showing his zuri turns away uncomfortable.James walks over and looks out of a peep hole, then turns back with a confused expression .Klaue spins toward the commotion as ...", 'Y The Last Man': "Track over to his face, which we can now see is wearing the familiar GAS The gasmask-clad Yorick looks around to make sure that no one is watching, before he reaches for one of the fallen Amazon's REVOLVERS.She pulls out her own .357 MAGNUM and looks around, finally noticing a CRATE that’s been recently opened, one large enough for two people to hide inside of.She's rolling a PAWN PIECE back and forth across her As a baby CRIES somewhere in the plane’s cabin, a pensive Beth is looking out her window at the world far below.Smash cut to a few hours later for this image of a hissing Yorick, as his eyes suddenly SPRING OPEN.Just then, a SCREECHING is heard, and the two soldiers pivot to see Ampersand at them from another column in the Before this bizarre sight has time to sink in with the two soldier, 355 takes advantage of the distraction to SHOOT the women dead.The woman turns on the flashlight to get a better look at the young man she just struck.The two look int", 'Shadow Dancer': "In the side street where MAC is standing -- barely fifty yards behind him -- a couple of guys in balaclavas are throwing home-made pipe bombs over the top of the cops and into the crowd.We hear the roar of the chamber... Hacks, officials...all stunned, paralysed...GERRY is slowly walking towards the front.... MAC jumps a desk, slides across a wide central table strewn with press releases and bursts through the door... .+.as GERRY raises his gun arm.He looks down at the water -- there is indeed a boat waiting on the far side -- then turns to face his pursuer.We see a man spinning around and moving away.He looked at the receiver and puts it slowly back onto the We hear a cry in the dark.MA shakes her GERRY’s son LIAM has been driving a remote controlled car around the floor and at this moment he’s so excited he knocks over the table GERRY's glass of beer has been standing on.Another sweeps his hand along a kitchen cabinet, tipping everything onto the floor.He tears arou", 'Secretariat': 'Then SECRETARIAT--majestic, rippling, fluid, both beautiful and ferocious--appears around the turn and flies onto the backstretch, Turcotte on his saddle, steady as a rock.We see WHAT SHE SEES IN HER MEMORY --A little girl--Penny, at 6--standing at the rail of a racetrack, beside her tall, handsome father--CHRISTOPHER CHENERY--in the prime of his life, as the crowd thunders --A pack of racehorses rounding a turn and heading into the home stretch, hooves pounding, muscles rippling, colors flashing, the horses thundering toward the wire.She pulls the camera down...and takes a step backwards toward her .the bugle call to post sounds...', 'Dunkirk': 'The Stretcher Bearer NODS, too breathless to speak, then follows his colleagues BACK DOWN OFF THE SHIP- An EXPLOSION hits the water nearby- Everyone hits the deck as shells IMPACT the water Tommy is PULLED UP SHORT as Gibson STUMBLES- A 109 STRAFES the length of the mole with gunfire- soldiers hit the deck, several are HIT... Peter and George RUSH things off the boat- then start loading the orange life preservers Mr.Dawson looks up from his charts to see NAVAL OFFICERS AND CREW coming along the harbor, Peter follows his gaze- Peter bursts into the cabin, stacking life vests- The pile of life vests on the dock SHRINKS...', 'Hold the Dark': "As he turns back into the road, Cheeon calls to him... Marium doesn’t turn back, keeps walking stiffly away, his jaw tight as Cheeon SLAMS the door behind him.He whips back to panic-frozen Arnie-- BEHIND ANOTHER TRUCK, Core is flat on the ground, looking out from underneath it, shaking, awestruck-- He sees DEAD AND WOUNDED COPS splayed before CHEEON’S HOUSE with MUZZLE FLASHES flaming out of the ATTIC WINDOW.Core pulls him down to some kind of safety behind some ROCKS, throwing his own body of Marium's, trying to press his gloves But Marium’s already dead, his eyes fixed and startled.A deep breath for courage and he eases it open but wait-- LOOKING OUT ON MARIUM THROUGH THE DOOR WINDW as he freezes, terrified eyes locked on something dead ahead.He coughs softly, rolls away from it... CLOSE ON CORE as he stares straight ahead at something else, something in n", 'Bridget Jones s Baby': 'This time Bridget pauses a moment too long.Bridget and Jack are together on the sofa watching Mark, distracted, at the boarding gate for KHARTOUM.As they walk away Bridget turns around and exchanges a smile works in the Starbucks in Balham.A good looking man clocks her and takes her in as she walks - she smiles to herself once he has passed.She stops behind a flower display and looks back at Mark patiently watching a film with five there, cows with opposable thumbs, We find Jude fast asleep in her hotel room, with the baby next to her.Bridgemt strides into the studio, ready for the day.studio, where Miranda is waiting Behind Bridget, we see Miranda, live in the studio , waiting We see the Hard News countdown clock on the screens and then Bridget holds up her phone and says “Good Evening Miranda”.Dad fights his way to the stage, phone in hand.Bridger steps back and looks down to her bump.bridget turns round to find somebody to take a photo of them.Both men are at the door, look', 'Dirty Girl': "Danielle plants a kiss on Greg, Greg goes along with the kiss, but is embarrassed in front of Danielle, having retrieved Greg's gum from his mouth, blows a Clarke, slowly walks away, then turns back.She seemed Danielle looks at Ray, rolls her eyes and heads to her room.Joseph pulls over to the side of the road and smacks Clarke's face, grabs him by the shirt, and pulls him into the front military academy is looking like a A knowing look from his mother, tells Clarke that agreeing with her will free him.", 'This Is the End': "Jay opens the bathroom door to find Michael Cera with one girl blowing him and another eating his ass.His eyes go wide, his jaw drops, and he screams like a little girl.Seth, Jay and Franco reach the gars getting cornered by the large, over to the guys and sees“ smiles, then looks back at 1 speed towards his doom.Jay looks to knife from the kitchen, and throws it laps, sending the guys CRASHING back into the house.They look up and see Seth and Jay take off running, as they smash into thing ING BACK TO FRANCO'S!Jay flicks the lighter again, and this time, the FLAME LIGHTS and we see WHAT APPEARS TO BE A MONSTER standing right beside them.The guys slowly WALK into the living room and wriggling around on the floor, his eyes roli Jay and Craig tip toe through the house, their weapons held high.They looked away away from trying the same thing.Thick black s Jayand Craig hold hands as they slowly make their way through the smoke.Jonah slams into the painting, hits the floor", 'Ryan Hansen Solves Crimes on Television': 'light smile lines coming in here and Mathers and Ryan talk with a scarf-wearing ALFONSO DIAPHANO (60s) who flutters about the space getting things ready for the big show tonight.Behind Ryan, a VAMPY GIRL in crazy skimpy booty shorts, holds her phone down below her ass then brings it up to her face where she smiles and gives a peace sign while holding a package of "Flat Tummy Tea" she\'s hawking.Reveal Jared, sporting a cast on his broken arm, shoves Ryan and Mather\'s DEEPER BACK STAGE, away from everyone, as Parker joins his side smiling smugly.Then we hear the cry... Parker spins just in time to take a crisp right cross to the jaw from Mathers -- CLUNK.Suddenly, Ryan starts jerk his body all over the', 'Pacific Rim': "She hears a deafening ROAR and turns to see another memory playing out in the water just off the Pier -- Two vast figures clashing in the surf washes over Flick's face as she watches.The shattered windows come back together as the shock wave lifts Young Mako off the floor, and the roar retreats, going back into Trespasser's Dread.Without turning on the lights she drags one out into the middle of the floor and strains to pry open the lid with her bare hands -- she doesn’t know what kind of time she has.Flick packs a suitcase, her eyes still red, her hands still shaking slightly.BACK TO PRESENT The NOISE of the memory echoes as Raleigh pulls himself up to sit on the edge of the bed, his face in his hands.They both chuckle awkwardly as they stare at one another across the hall, a thought on the tip of their tongues, Simultaneously, they realize there's no point in trying to phrase it, and they let themselves collide in the middle hallway, kissing hungrily.An AIR R"}
summary_large_dict

{'The Guard': "OVERHEAD SHOT -- BOYLE leans back into the pillows and sighs, a big grin on his face.Then looks back at BOYLLE.Gets out, looks around -- TWO LITTLE GIRLS are looking on.EVERETT watches her go, then looks at STANTON, who is still seated.They look blankly at one another.They are blankly up at ONE another.Then at the BOYLES, who are standing nearby, playing with the big toe of another corpse, a blank look on their face.He turns around -- O'LEARY is sitting in an armchair, a gun in one hand, pointed at BOYLE, a whiskey in the other.BOYLE enters and looks around, grinning -- A big map.EVEREET looks on, impressed, as the MAN turns towards the strand and swims in.As CORNELL walks off, he makes a sinister turn and looks backat STANTON and MOODY, giving them pause.He and SHEEHY look at one other, until SHEEhy becomes aware of the song that is playing --  BOY LE shoots a glance at AOIFE.Then he hears a sound and looks up -- ’BOYLE’s",
 'Batman Year One': "While FLASS looks at the 

In [ ]:
two_stage_large_results = combine_rouge_and_summary_results(rouge_large_dict, summary_large_dict, 'two_stage_large')
two_stage_large_results.head()

,rouge1,rouge2,rougeL,rougeLsum,two_stage_large_summary
The Guard,0.133929,0.009009,0.098214,0.098214,OVERHEAD SHOT -- BOYLE leans back into the pil...
Batman Year One,0.200669,0.013468,0.113712,0.113712,"While FLASS looks at the cuffs, FLASS, angry, ..."
Inherent Vice,0.139535,0.000000,0.124031,0.124031,DOC gives him a PHOTO OF COY like The Last Sup...
Colombiana,0.170732,0.000000,0.105691,0.105691,"She flattens against the wall, holding onto th..."
Barney s Version,0.141304,0.010989,0.097826,0.097826,"Off Barney, completely shell shocked we -- PRE..."


In [ ]:
# save results to a csv
two_stage_large_results.to_csv('/content/two_stage_text_large_results.csv')
# files.download('/content/two_stage_large_results.csv') 

## Two Stage Long T5 Model
This model combines an extractive summary using TextRank, which is then used for abstractive summary using the *LongT5* model

In [ ]:
rouge_t5_dict = {}
summary_t5_dict = {}

rouge_t5_scores, summaries_t5 = execute_two_stage_text_t5_model(0, 30, rouge_t5_dict, summary_t5_dict, subset_train_df)

print(rouge_t5_scores)
print(summaries_t5)

846


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('OVERHEAD SHOT -- BOYLE leans back into the pillows and sighs, a big grin on '
 'his face.Then looks back at BOYLLE.Gets out, looks around -- TWO LITTLE '
 'GIRLS are looking on.EVERETT watches her go, then looks at STANTON, who is '
 'still seated.They look blankly at one another.They are blankly up at ONE '
 'another.Then at the BOYLES, who are standing nearby, playing with the big '
 "toe of another corpse, a blank look on their face.He turns around -- O'LEARY "
 'is sitting in an armchair, a gun in one hand, pointed at BOYLE, a whiskey in '
 'the other.BOYLE enters and looks around, grinning -- A big map.EVEREET looks '
 'on, impressed, as the MAN turns towards the strand and swims in.As CORNELL '
 'walks off, he makes a sinister turn and looks backat STANTON and MOODY, '
 'giving them pause.He and SHEEHY look at one other, until SHEEhy becomes '
 'aware of the song that is playing --  BOY LE shoots a glance at AOIFE.Then '
 'he hears a sound and looks up -- ’BOYLE’s')
1112


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('While FLASS looks at the cuffs, FLASS, angry, throws them back and SLAMS the '
 'CLOSET DOOR  Just at the moment BRUCE CRASHES into the rat-infested GORDON '
 'hearing something, takes a look out the window where BRUCE was hanging '
 'moments ago.Upstairs, the SWAT TEAM, guns poised, peeks over the lip of the '
 'broken ceiling just in time to get a face full of  Outside the BATHROOM '
 'door, THE BAT-MAN covers his ears until the debris settles.But he COMES UP '
 'SHOOTING DOUBLE HEAD SHOT  blood sprays, one down  TWO CHEST SHOTS  a second '
 'cop staggers back, stricken and ONE IN THE HIP  third cop spins, but manages '
 'to keep his feet, until  BLAM!He hangs four stories up by his fingers as '
 'inside  Then, confronted with the sight of his DEAD PARTNER  Out the window '
 'BRUCE is slipping.Once she is close enough to the guards, she whips off her  '
 'the first one across the face with the buckle, knocking his  her long nails '
 "across the second's face.After a moment BAVEN st

...

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('As he turns back into the road, Cheeon calls to him... Marium doesn’t turn '
 'back, keeps walking stiffly away, his jaw tight as Cheeon SLAMS the door '
 'behind him.He whips back to panic-frozen Arnie-- BEHIND ANOTHER TRUCK, Core '
 'is flat on the ground, looking out from underneath it, shaking, awestruck-- '
 'He sees DEAD AND WOUNDED COPS splayed before CHEEON’S HOUSE with MUZZLE '
 'FLASHES flaming out of the ATTIC WINDOW.Core pulls him down to some kind of '
 "safety behind some ROCKS, throwing his own body of Marium's, trying to press "
 'his gloves But Marium’s already dead, his eyes fixed and startled.A deep '
 'breath for courage and he eases it open but wait-- LOOKING OUT ON MARIUM '
 'THROUGH THE DOOR WINDW as he freezes, terrified eyes locked on something '
 'dead ahead.He coughs softly, rolls away from it... CLOSE ON CORE as he '
 'stares straight ahead at something else, something in n')
1178


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('This time Bridget pauses a moment too long.Bridget and Jack are together on '
 'the sofa watching Mark, distracted, at the boarding gate for KHARTOUM.As '
 'they walk away Bridget turns around and exchanges a smile works in the '
 'Starbucks in Balham.A good looking man clocks her and takes her in as she '
 'walks - she smiles to herself once he has passed.She stops behind a flower '
 'display and looks back at Mark patiently watching a film with five there, '
 'cows with opposable thumbs, We find Jude fast asleep in her hotel room, with '
 'the baby next to her.Bridgemt strides into the studio, ready for the '
 'day.studio, where Miranda is waiting Behind Bridget, we see Miranda, live in '
 'the studio , waiting We see the Hard News countdown clock on the screens and '
 'then Bridget holds up her phone and says “Good Evening Miranda”.Dad fights '
 'his way to the stage, phone in hand.Bridger steps back and looks down to her '
 'bump.bridget turns round to find somebody to take a pho

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
two_stage_t5_results = combine_rouge_and_summary_results(rouge_t5_dict, summary_t5_dict, 'two_stage_t5')
two_stage_t5_results.head()

In [ ]:
# save results to a csv
two_stage_t5_results.to_csv('/content/two_stage_text_t5_results.csv')
# files.download('/content/two_stage_text_t5_results.csv') 

In [ ]:
# average rouge scores
two_stage_t5_results.drop(columns=['two_stage_t5_summary']).mean(axis=0)

# TextRank Sentences (BERT Dialog)

In [ ]:
ranked_sentences_dialog_dict = {}

for i in range(30):
    ranked_sentences = run_textrank_model(subset_train_df.iloc[i]['bert_dialog'])
    
    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    ranked_sentences_dialog_dict[title] = ranked_sentences

ranked_sentences_dialog_dict

1701
1480
1807
1467
1736
1702
1173
1458
1366
2077
1597
1322
1833
1062
1009
1643
1440
1115
1587
1531
1417
1578
1446
510
1083
1672
1471
1478
594
1415


{'The Guard': "You know nothing about me.(getting worked up)\n Well why don’t you put in your report\n that Sergeant Boyle went out of his\n fucking way to do you boys a fucking\n favour and he got really fucking\n annoyed when you started asking him\n stupid fucking questions about a few\n missing fucking guns, trying to\n fucking catch him out as if he’s just\n some kind of fucking gobshite!I’m not sure if you ever get over\n something like that.I just didn’t realise...\n Do you think he might have met\n someone here who did something bad\n to him?Pretending to be sad is what\n they pay me for, I always think.But I cannot think of anybody who\n would have wanted to do something\n bad to Aidan.Are you going to continue to make\n or are you going to fucking tell\n me something?Sure don’t I know well.We all know that.you know.You know.You know him?You know him?You know for what.Now do I know him?I know.I know.I know.I know.I know.87\n 87\n I know.I know you weren’t.Sure I know well.You 

In [ ]:
textrank_dialog_df = pd.DataFrame(ranked_sentences_dialog_dict.items(), columns=['title', 'ranked_sentences'])
textrank_dialog_df.head()

,title,ranked_sentences
0,The Guard,You know nothing about me.(getting worked up)\...
1,Batman Year One,"Sooner or\n later, we'll get lucky and someone..."
2,Inherent Vice,Said Shasta was the only one that\n ever made ...
3,Colombiana,You know what time it\n is?We'll get all this ...
4,Barney s Version,"There's so much I want\n to talk about... Oh, ..."


In [ ]:
# save results to a csv
textrank_dialog_df.to_csv('/content/textrank_dialog.csv')
# files.download('/content/textrank_dialog.csv') 

# TextRank Sentences (BERT Text)

In [ ]:
ranked_sentences_text_dict = {}

for i in range(30):
    ranked_sentences = run_textrank_model(subset_train_df.iloc[i]['bert_text'])
    
    # add rouge scores and summary to a dictionary
    title = subset_train_df.iloc[i]['title']
    ranked_sentences_text_dict[title] = ranked_sentences

ranked_sentences_text_dict

846
1112
374
1524
1007
849
1416
635
953
842
899
844
1189
1028
1549
1149
732
1041
637
969
776
1434
843
549
1378
1178
634
622
251
1717


{'The Guard': "OVERHEAD SHOT -- BOYLE leans back into the pillows and\n sighs, a big grin on his face.Then looks back at\n BOYLE.Gets out, looks around --\n TWO LITTLE GIRLS are looking on.EVERETT watches her go, then looks\n at STANTON, who is still seated.They look blankly at one another.They look blankly at one another.Then at\n BOYLE, who is standing nearby, playing with the big toe\n of another corpse, a blank look on his face.He turns around --\n O'LEARY is sitting in an armchair, a gun in one hand,\n pointed at BOYLE, a whiskey in the other.BOYLE enters and looks around, grinning --\n A big map.EVERETT looks on, impressed, as the MAN turns towards\n the strand and swims in.As CORNELL walks off, he makes a sinister turn and looks\n back at STANTON and MOODY, giving them pause.He and SHEEHY look at one another, until\n SHEEHY becomes aware of the song that is playing --\n BOYLE shoots a glance at AOIFE.Then he hears a sound and looks up --\n BOYLE’s POV -- a car is coasting down t

In [ ]:
textrank_text_df = pd.DataFrame(ranked_sentences_text_dict.items(), columns=['title', 'ranked_sentences'])
textrank_text_df.head()

,title,ranked_sentences
0,The Guard,OVERHEAD SHOT -- BOYLE leans back into the pil...
1,Batman Year One,"While FLASS looks at the cuffs,\n FLASS, angry..."
2,Inherent Vice,DOC gives him a PHOTO OF COY like The Last Sup...
3,Colombiana,"She flattens against the wall, holding onto th..."
4,Barney s Version,"Off Barney, completely shell shocked we --\n P..."


In [ ]:
# save results to a csv
textrank_text_df.to_csv('/content/textrank_text.csv')
# files.download('/content/textrank_text.csv') 

# Avatar -- Summary Comparison

### Movie script

In [ ]:
# get the movie info for Avatar
avatar_info = data_df.loc[data_df['title'] == 'Avatar']

# drop leading zeros for imdb_id
avatar_info['imdb_id'] = avatar_info['imdb_id'].str.lstrip('0')

# merge with wiki reference
avatar_df = pd.merge(avatar_info, wiki_df, how='outer', on='imdb_id')

# rename columns
avatar_df = avatar_df.rename(columns={'title_x': 'title'})

# only keep the first row
avatar = avatar_df.head(1)

# get bert dialog
avatar['bert_dialog'] = avatar.apply(lambda x: concatenate_dialog(x['bert_data']), axis=1)

# get bert text
avatar['bert_text'] = avatar.apply(lambda x: concatenate_text(x['bert_data']), axis=1)

# reorder columns
avatar = avatar[['title', 'imdb_id', 'bert_data', 'lemma_data', 'plot_outline', 'bert_dialog', 'bert_text']]

avatar

<ipython-input-40-cd1d304a32ac>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avatar_info['imdb_id'] = avatar_info['imdb_id'].str.lstrip('0')
<ipython-input-40-cd1d304a32ac>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avatar['bert_dialog'] = avatar.apply(lambda x: concatenate_dialog(x['bert_data']), axis=1)
<ipython-input-40-cd1d304a32ac>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,title,imdb_id,bert_data,lemma_data,plot_outline,bert_dialog,bert_text
0,Avatar,499549,/content/drive/MyDrive/W266_Movie_Data/BERT_an...,/content/drive/MyDrive/W266_Movie_Data/raw_tex...,"When his brother is killed in a robbery, parap...",\n \n \n \n \n \n \n Written by\n \n James Ca...,"THE SOUND OF DRUMS, from a great distance, gr..."


In [ ]:
# store all summary information and their rouge scores
avatar_summary_dict = {}
avatar_rouge_dict = {}

## Lemmas

Pegasus Short

In [ ]:
raw_lemma = avatar['lemma_data'][0]

with open(raw_lemma, 'r') as file:
    text_data = file.read().replace('\n', '')

summary = run_pegasus_short_model(text_data)
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['lemma_pegasus_short'] = summary
avatar_rouge_dict['lemma_pegasus_short'] = rouge_scores

print(summary)
print(rouge_scores)

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("Jake's voice is so strong it's hard to hear him , but I can see it in his "
 "eyes and his face and his eyes can't see me , so I can hear him in my head "
 'and my head can see him in his body and his body can see me in my body and '
 'my body can hear me in his mind and my mind can hear us in our body and our '
 "mind and we can hear each other's voices.")
Jake's voice is so strong it's hard to hear him , but I can see it in his eyes and his face and his eyes can't see me , so I can hear him in my head and my head can see him in his body and his body can see me in my body and my body can hear me in his mind and my mind can hear us in our body and our mind and we can hear each other's voices.
{'rouge1': 0.14225941422594143, 'rouge2': 0.0, 'rougeL': 0.10041841004184099, 'rougeLsum': 0.10041841004184099}


Pegasus Long

In [ ]:
raw_lemma = avatar['lemma_data'][0]

with open(raw_lemma, 'r') as file:
    text_data = file.read().replace('\n', '')

summary = run_pegasus_long_model(text_data)
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['lemma_pegasus_long'] = summary
avatar_rouge_dict['lemma_pegasus_long'] = rouge_scores

print(summary)
print(rouge_scores)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Combat vet Jake sully has been injured on duty in Iraq and Afghanistan '
 '.<n>He now lives in a wheelchair in a VA hospital in NYC and struggles to '
 'cope with his injuries ..<n>The show follows Jake as he tries to come to '
 'terms with his disability and the challenges he faces in getting back on his '
 "feet . Hilarious and touching performances from the show's cast and crew, "
 "including Jake's best friend, Kate who is also a vet .")
Combat vet Jake sully has been injured on duty in Iraq and Afghanistan .<n>He now lives in a wheelchair in a VA hospital in NYC and struggles to cope with his injuries ..<n>The show follows Jake as he tries to come to terms with his disability and the challenges he faces in getting back on his feet . Hilarious and touching performances from the show's cast and crew, including Jake's best friend, Kate who is also a vet .
{'rouge1': 0.2638297872340426, 'rouge2': 0.034334763948497854, 'rougeL': 0.15319148936170213, 'rougeLsum': 0.15319148936170213}


Pegasus Large

In [ ]:
raw_lemma = avatar['lemma_data'][0]

with open(raw_lemma, 'r') as file:
    text_data = file.read().replace('\n', '')

summary = run_pegasus_large_model(text_data)
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['lemma_pegasus_large'] = summary
avatar_rouge_dict['lemma_pegasus_large'] = rouge_scores

print(summary)
print(rouge_scores)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('we be very low over the forest now , glide fast , the drum build to a peak '
 '-- voice ( v.o. CITY - night a screech of brake as a vehicle wipe frame , '
 'revealing -- jake sully , a scar and scruffy combat vet , sit in a beat up '
 'carbon - fiber wheelchair . Jake stare upward at the level of the city . ) '
 'they can fix a spinal , if you have get the money .')
we be very low over the forest now , glide fast , the drum build to a peak -- voice ( v.o. CITY - night a screech of brake as a vehicle wipe frame , revealing -- jake sully , a scar and scruffy combat vet , sit in a beat up carbon - fiber wheelchair . Jake stare upward at the level of the city . ) they can fix a spinal , if you have get the money .
{'rouge1': 0.16289592760180996, 'rouge2': 0.027397260273972605, 'rougeL': 0.09954751131221719, 'rougeLsum': 0.09954751131221719}


Long T5

In [ ]:
raw_lemma = avatar['lemma_data'][0]

with open(raw_lemma, 'r') as file:
    text_data = file.read().replace('\n', '')

summary = run_longt5_model(text_data)
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['lemma_longt5'] = summary
avatar_rouge_dict['lemma_longt5'] = rouge_scores

print(summary)
print(rouge_scores)

/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:784: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


('The narrator tells us that this is one of the most famous lines in the play. '
 "It's about a drum roll, which sounds like it's coming from somewhere high up "
 "in the sky. You can hear it when you're lying down in the hospital and you "
 'wake up with a huge blow through your heart. This is kind of like watching a '
 "horror movie on repeat. In other words, there's a lot of scary stuff going "
 'on here.')
The narrator tells us that this is one of the most famous lines in the play. It's about a drum roll, which sounds like it's coming from somewhere high up in the sky. You can hear it when you're lying down in the hospital and you wake up with a huge blow through your heart. This is kind of like watching a horror movie on repeat. In other words, there's a lot of scary stuff going on here.
{'rouge1': 0.19491525423728814, 'rouge2': 0.008547008547008546, 'rougeL': 0.11864406779661017, 'rougeLsum': 0.11864406779661017}


## BERT - Dialog

Pegasus Short

In [ ]:
summary = run_pegasus_short_model(avatar['bert_dialog'][0])
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['dialog_pegasus_short'] = summary
avatar_rouge_dict['dialog_pegasus_short'] = rouge_scores

print(summary)
print(rouge_scores)

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("James Cameron's latest film, The Lone Ranger, opens in US cinemas on Friday, "
 "8 March and in UK cinemas on Monday, 9 April..... and here's a song from the "
 'film, written and directed by James Cameron, featuring the cast of '
 "characters from the movie, including Forest Whitaker, Lupita Nyong'o, and "
 "Forest Whitaker's wife, Jennifer Lawrence.. and the film's soundtrack, which "
 'includes music from the Oscar-winning director James Cameron..')
James Cameron's latest film, The Lone Ranger, opens in US cinemas on Friday, 8 March and in UK cinemas on Monday, 9 April..... and here's a song from the film, written and directed by James Cameron, featuring the cast of characters from the movie, including Forest Whitaker, Lupita Nyong'o, and Forest Whitaker's wife, Jennifer Lawrence.. and the film's soundtrack, which includes music from the Oscar-winning director James Cameron..
{'rouge1': 0.14035087719298245, 'rouge2': 0.0, 'rougeL': 0.11403508771929824, 'rougeLsum': 0.1140350877192

Pegasus Long

In [ ]:
summary = run_pegasus_long_model(avatar['bert_dialog'][0])
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['dialog_pegasus_long'] = summary
avatar_rouge_dict['dialog_pegasus_long'] = rouge_scores

print(summary)
print(rouge_scores)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


("James Cameron's new song 'I Don't Want Your pity' is out now on iTunes "
 ".<n>It's a response to President Obama's speech at last week's G-8 summit in "
 'L.A.<n>The song was written while Cameron was lying in the VA during the '
 "middle of his life ..<n>Cameron: 'They can fix a spinal, if you've got the "
 'money. But not on vet benefits, not in this economy. isolated souls. '
 "century, is making a comeback'")
James Cameron's new song 'I Don't Want Your pity' is out now on iTunes .<n>It's a response to President Obama's speech at last week's G-8 summit in L.A.<n>The song was written while Cameron was lying in the VA during the middle of his life ..<n>Cameron: 'They can fix a spinal, if you've got the money. But not on vet benefits, not in this economy. isolated souls. century, is making a comeback'
{'rouge1': 0.16033755274261602, 'rouge2': 0.0, 'rougeL': 0.10970464135021098, 'rougeLsum': 0.10970464135021098}


Pegasus Large

In [ ]:
summary = run_pegasus_large_model(avatar['bert_dialog'][0])
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['dialog_pegasus_large'] = summary
avatar_rouge_dict['dialog_pegasus_large'] = rouge_scores

print(summary)
print(rouge_scores)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Sooner or later though, you always have to wake up... pain beyond his years. '
 'All I ever wanted in my sorry-ass life was a single thing worth fighting '
 "for. I don't want your pain. I know the world's a cold ass bitch. You want a "
 "fair deal, you're on the wrong chair. And nobody does a damn thing. I told "
 'myself I could pass any test a man can pass. To be hammered on the anvil of '
 'life.')
Sooner or later though, you always have to wake up... pain beyond his years. All I ever wanted in my sorry-ass life was a single thing worth fighting for. I don't want your pain. I know the world's a cold ass bitch. You want a fair deal, you're on the wrong chair. And nobody does a damn thing. I told myself I could pass any test a man can pass. To be hammered on the anvil of life.
{'rouge1': 0.13559322033898308, 'rouge2': 0.008547008547008546, 'rougeL': 0.10169491525423728, 'rougeLsum': 0.10169491525423728}


LongT5

In [ ]:
summary = run_longt5_model(avatar['bert_dialog'][0])
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['dialog_longt5'] = summary
avatar_rouge_dict['dialog_longt5'] = rouge_scores

print(summary)
print(rouge_scores)

Your max_length is set to 256, but you input_length is only 248. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=124)


('In this chapter, the narrator explains how he came to be in the middle of '
 'his life when he was lying in the hospital. He dreamed of flying and '
 "becoming a soldier. But now that he's out of the hospital, he realizes what "
 "it's like to be an isolated soul. He decides to become a Marine so he can "
 'deal with the "anvil of life" .')
In this chapter, the narrator explains how he came to be in the middle of his life when he was lying in the hospital. He dreamed of flying and becoming a soldier. But now that he's out of the hospital, he realizes what it's like to be an isolated soul. He decides to become a Marine so he can deal with the "anvil of life" .
{'rouge1': 0.2533936651583711, 'rouge2': 0.027397260273972605, 'rougeL': 0.15384615384615385, 'rougeLsum': 0.15384615384615385}


## BERT - Text

Pegasus Short

In [ ]:
summary = run_pegasus_short_model(avatar['bert_text'][0])
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['text_pegasus_short'] = summary
avatar_rouge_dict['text_pegasus_short'] = rouge_scores

print(summary)
print(rouge_scores)

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Jake is a young man in his early 20s who has just been released from prison '
 'and is about to embark on a new life in a small town in the American state '
 'of New Mexico, where he will live out his childhood dream of becoming a '
 'police officer, working alongside his father, who has been killed in the '
 'line of duty, and his mother, who is pregnant with his first child, and who '
 'is struggling to make ends meet as a single mother in a town with a high '
 'crime rate.')
Jake is a young man in his early 20s who has just been released from prison and is about to embark on a new life in a small town in the American state of New Mexico, where he will live out his childhood dream of becoming a police officer, working alongside his father, who has been killed in the line of duty, and his mother, who is pregnant with his first child, and who is struggling to make ends meet as a single mother in a town with a high crime rate.
{'rouge1': 0.2439024390243902, 'rouge2': 0.032786885245901

Pegasus Long

In [ ]:
summary = run_pegasus_long_model(avatar['bert_text'][0])
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['text_pegasus_long'] = summary
avatar_rouge_dict['text_pegasus_long'] = rouge_scores

print(summary)
print(rouge_scores)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('A scarred and scruffy combat vet, sitting in a beat up carbon-fiber '
 "wheelchair .<n>Jake's eyes are hardened by the wisdom and wariness of one "
 'who has endured Jake stares upward at the levels of the city.<n>The room is '
 'a tiny CCLE, prison cell meets 747 bathroom.<n>Narrow cot, wall-screen drums '
 'droning away in the B.G. -- The Bengal tiger, extinct for over over.<n>Most '
 'people wear KSKS to protect them from the toxic air.')
A scarred and scruffy combat vet, sitting in a beat up carbon-fiber wheelchair .<n>Jake's eyes are hardened by the wisdom and wariness of one who has endured Jake stares upward at the levels of the city.<n>The room is a tiny CCLE, prison cell meets 747 bathroom.<n>Narrow cot, wall-screen drums droning away in the B.G. -- The Bengal tiger, extinct for over over.<n>Most people wear KSKS to protect them from the toxic air.
{'rouge1': 0.20338983050847456, 'rouge2': 0.017094017094017092, 'rougeL': 0.1271186440677966, 'rougeLsum': 0.1271186440677966}


Pegasus Large

In [ ]:
summary = run_pegasus_large_model(avatar['bert_text'][0])
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['text_pegasus_large'] = summary
avatar_rouge_dict['text_pegasus_large'] = rouge_scores

print(summary)
print(rouge_scores)

All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('hospital, with a big hole blown through We are very low over the forest now, '
 'gliding fast, the drums to a PEAK -- A SCREECH OF BRAKES as a vehicle WIPES '
 'FRAME, revealing -- JAKE SULLY, a scarred and scruffy combat vet, sitting in '
 'a beat up carbon-fiber wheelchair. At 22, his eyes are hardened by the '
 'wisdom and wariness of one who has endured Jake stares upward at the levels '
 'of the city.')
hospital, with a big hole blown through We are very low over the forest now, gliding fast, the drums to a PEAK -- A SCREECH OF BRAKES as a vehicle WIPES FRAME, revealing -- JAKE SULLY, a scarred and scruffy combat vet, sitting in a beat up carbon-fiber wheelchair. At 22, his eyes are hardened by the wisdom and wariness of one who has endured Jake stares upward at the levels of the city.
{'rouge1': 0.18502202643171806, 'rouge2': 0.026666666666666665, 'rougeL': 0.11453744493392068, 'rougeLsum': 0.11453744493392068}


LongT5

In [ ]:
summary = run_longt5_model(avatar['bert_text'][0])
rouge_scores = calculate_rouge_scores(summary, avatar['plot_outline'])

# add rouge scores and summary to a dictionary
avatar_summary_dict['text_longt5'] = summary
avatar_rouge_dict['text_longt5'] = rouge_scores

print(summary)
print(rouge_scores)

('The sound of drums is heard from a distance. We are flying through mist and '
 'over the forest below. There is a hospital with a hole blown in, and we are '
 'glided by the drums building to a peak. A screech of brakes as a vechicle '
 'whines frammed; Jake Sully, a Scarred and Scuffy Combat Vet, sits in a '
 'beating up carbon fiber wheelchair. At 22 he eyes hardened by his wisdom '
 'andwariness of one whom has endured such a thing. Maglev Trails Whoosh '
 'overhead on raised tracks against a garish advertisementizing')
The sound of drums is heard from a distance. We are flying through mist and over the forest below. There is a hospital with a hole blown in, and we are glided by the drums building to a peak. A screech of brakes as a vechicle whines frammed; Jake Sully, a Scarred and Scuffy Combat Vet, sits in a beating up carbon fiber wheelchair. At 22 he eyes hardened by his wisdom andwariness of one whom has endured such a thing. Maglev Trails Whoosh overhead on raised tracks ag

## Two Stage BERT - Dialog

Short

In [ ]:
rouge_scores, summary = execute_two_stage_short_model(0, 1, avatar_rouge_dict, avatar_summary_dict, avatar)

# add rouge scores and summary to a dictionary
avatar_summary_dict['two_stage_dialog_pegasus_short'] = summary
avatar_rouge_dict['two_stage_dialog_pegasus_short'] = rouge_scores

print(summary)
print(rouge_scores)

1459


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('The Great People have sent a message that they can take whatever they want, '
 "and want one can stop them, but one can't stop them because they only come "
 'and make problems, so one day I could bring this message, and you would '
 'never be one of them, because one day you could become one of me, and I '
 'would never come back, and never come again, because the Sky has sent this '
 'message to you, so I have to go all the way to become one.')


{'Avatar': "The Great People have sent a message that they can take whatever they want, and want one can stop them, but one can't stop them because they only come and make problems, so one day I could bring this message, and you would never be one of them, because one day you could become one of me, and I would never come back, and never come again, because the Sky has sent this message to you, so I have to go all the way to become one.", 'two_stage_dialog_pegasus_short': {...}}
{'Avatar': {'rouge1': 0.13223140495867772, 'rouge2': 0.0, 'rougeL': 0.08264462809917356, 'rougeLsum': 0.08264462809917356}, 'two_stage_dialog_pegasus_short': {...}}


Long

In [ ]:
rouge_scores, summary = execute_two_stage_long_model(0, 1, avatar_rouge_dict, avatar_summary_dict, avatar)

# add rouge scores and summary to a dictionary
avatar_summary_dict['two_stage_dialog_pegasus_long'] = summary
avatar_rouge_dict['two_stage_dialog_pegasus_long'] = rouge_scores

print(summary)
print(rouge_scores)

1459


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Goddammit, Jake, you can never be one of them!<n>The Sky People have sent a '
 'message that they can take whatever they want, and no one can stop '
 'them.<n>The Great Mother may choose to save all that she is -- -- in this '
 "body.It's time to come in.<n>Nothing like an old-school safety brief to put "
 'your mind at ease.<n>And you may choose a woman.Neytiri, you know m')
{'Avatar': "Goddammit, Jake, you can never be one of them!<n>The Sky People have sent a message that they can take whatever they want, and no one can stop them.<n>The Great Mother may choose to save all that she is -- -- in this body.It's time to come in.<n>Nothing like an old-school safety brief to put your mind at ease.<n>And you may choose a woman.Neytiri, you know m", 'two_stage_dialog_pegasus_short': {...}, 'lemma_pegasus_short': "Jake's voice is so strong it's hard to hear him , but I can see it in his eyes and his face and his eyes can't see me , so I can hear him in my head and my head can see him in his

Large

In [ ]:
rouge_scores, summary = execute_two_stage_large_model(0, 1, avatar_rouge_dict, avatar_summary_dict, avatar)

# add rouge scores and summary to a dictionary
avatar_summary_dict['two_stage_dialog_pegasus_large'] = summary
avatar_rouge_dict['two_stage_dialog_pegasus_large'] = rouge_scores

print(summary)
print(rouge_scores)

1459


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('He was the one who wanted to get shot light years out 5.You only come and '
 'make problems.So one day I could bring this message, and you would believe '
 'it.I have to go all the way -- become one of Goddammit, Jake, you can never '
 'be one of them!All I ever wanted in my sorry-ass life was a single thing '
 'worth fighting for.Where ever you are, come back to me now.The Sky People '
 'have sent a message that they can take whatever they want, and no one can '
 "stop them.Get me what I need, I'll see you get your legs back when you "
 'rotate home.Never come back!The Great Mother may choose to save all that she '
 "is -- -- in this body.It's time to come in.Nothing like an old-school safety "
 "brief to put your mind at ease.Since a deal can't be made -- it gets real "
 "simple.So you just figured you'd come out here to the most hostile "
 'environment known to man, with no training of any kind, and see how it '
 'went?You may make your (she looks away) And you may choose a woman.

LongT5

In [ ]:
rouge_scores, summary = execute_two_stage_t5_model(0, 1, avatar_rouge_dict, avatar_summary_dict, avatar)

# add rouge scores and summary to a dictionary
avatar_summary_dict['two_stage_dialog_longt5'] = summary
avatar_rouge_dict['two_stage_dialog_longt5'] = rouge_scores

print(summary)
print(rouge_scores)

1459


/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:784: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


('The narrator tells us that Neytiri is the only person who has ever wanted to '
 "be shot out of the sky. He's been waiting for this message for years, and "
 "now it's time for him to come back. No one can stop the sky people from "
 "taking whatever they want. They're going to save everything in her body--and "
 "she's going to take care of herself. That's right, there's nothing like an "
 'old school brief to put you at ease.')
{'Avatar': "The narrator tells us that Neytiri is the only person who has ever wanted to be shot out of the sky. He's been waiting for this message for years, and now it's time for him to come back. No one can stop the sky people from taking whatever they want. They're going to save everything in her body--and she's going to take care of herself. That's right, there's nothing like an old school brief to put you at ease.", 'two_stage_dialog_pegasus_short': {...}, 'lemma_pegasus_short': "Jake's voice is so strong it's hard to hear him , but I can see it in his

## Two Stage BERT - Text

Short

In [ ]:
rouge_scores, summary = execute_two_stage_text_short_model(0, 1, avatar_rouge_dict, avatar_summary_dict, avatar)

# add rouge scores and summary to a dictionary
avatar_summary_dict['two_stage_text_pegasus_short'] = summary
avatar_rouge_dict['two_stage_text_pegasus_short'] = rouge_scores

print(summary)
print(rouge_scores)

2083


All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('All photographs courtesy of Getty Images, EPA, Getty Images and Reuters, '
 'subject of copyright laws and subject to change at any time without prior '
 'notice.,  Copyright (c) The Vancouver Sun E-mail this Article to a friend or '
 '(250 words or more) Postmedia News (Vancouver, British Columbia, Canada) '
 'Twitter: www.twitter.com/thesun Vancouver Sun (British Columbia) Instagram: '
 'www.99thesun.tumblr.com (Vancouver)')
{'Avatar': 'All photographs courtesy of Getty Images, EPA, Getty Images and Reuters, subject of copyright laws and subject to change at any time without prior notice.,  Copyright (c) The Vancouver Sun E-mail this Article to a friend or (250 words or more) Postmedia News (Vancouver, British Columbia, Canada) Twitter: www.twitter.com/thesun Vancouver Sun (British Columbia) Instagram: www.99thesun.tumblr.com (Vancouver)', 'two_stage_dialog_pegasus_short': {...}, 'lemma_pegasus_short': "Jake's voice is so strong it's hard to hear him , but I can see it in his eyes 

Long

In [ ]:
rouge_scores, summary = execute_two_stage_text_long_model(0, 1, avatar_rouge_dict, avatar_summary_dict, avatar)

# add rouge scores and summary to a dictionary
avatar_summary_dict['two_stage_text_pegasus_long'] = summary
avatar_rouge_dict['two_stage_text_pegasus_long'] = rouge_scores

print(summary)
print(rouge_scores)

2083


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('Jake runs in a blur, dodging between trunks as a glistening black tornado '
 "shreds the forest behind him .<n>The thanator's jaws SHUT SNAP inches behind "
 'him as he flies out into open space and -- JAKE SPLASHIVES down into a '
 'swiftly moving river ..<n>SNARLING, a wounded wolf attacks Jake, and he '
 'KICKS it away, It SPINS and leaps back onto him, and Jake barely catches its '
 'throat in time to keep the SNAPPING JAWS away from his face .')
{'Avatar': "Jake runs in a blur, dodging between trunks as a glistening black tornado shreds the forest behind him .<n>The thanator's jaws SHUT SNAP inches behind him as he flies out into open space and -- JAKE SPLASHIVES down into a swiftly moving river ..<n>SNARLING, a wounded wolf attacks Jake, and he KICKS it away, It SPINS and leaps back onto him, and Jake barely catches its throat in time to keep the SNAPPING JAWS away from his face .", 'two_stage_dialog_pegasus_short': {...}, 'lemma_pegasus_short': "Jake's voice is so strong it's 

Large

In [ ]:
rouge_scores, summary = execute_two_stage_text_large_model(0, 1, avatar_rouge_dict, avatar_summary_dict, avatar)

# add rouge scores and summary to a dictionary
avatar_summary_dict['two_stage_text_pegasus_large'] = summary
avatar_rouge_dict['two_stage_text_pegasus_large'] = rouge_scores

print(summary)
print(rouge_scores)

2083


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('JAKE RUNS in a blur, dodging between trunks as a glistening black tornado '
 'shreds the forest behind him and -- He sees WATER ahead and DIVES OUTWARD '
 "with all his might -- The thanator's jaws SHUT inches behind him as he flies "
 'out into open space and -- JAKE SPLASHES down into a swiftly moving river.He '
 'just lies there, blinking -- then shouts jauntily to no one in particular -- '
 'CAMERA PULLS BACK high and wide, as Jake lies spread-eagled amongst the '
 'trash, getting drenched.SNARLING, a wounded wolf attacks Jake, and he KICKS '
 'it away, It SPINS and leaps back onto him, and Jake barely catches its '
 'throat in time to keep the SNAPPING JAWS away from his face.TIME CUT -- '
 'Grace is scanning through images and Jake stops her on one -- a 3D aerial '
 'shot of the strange arched She moves the virtual camera, and we seem to fly '
 'around the Well of Souls, catching only a glimpse of the interior.CAMERA '
 'SCREAMS down on him as the shadow grows larger -- WE RUSH

LongT5

In [ ]:
rouge_scores, summary = execute_two_stage_text_t5_model(0, 1, avatar_rouge_dict, avatar_summary_dict, avatar)

# add rouge scores and summary to a dictionary
avatar_summary_dict['two_stage_text_longt5'] = summary
avatar_rouge_dict['two_stage_text_longt5'] = rouge_scores

print(summary)
print(rouge_scores)

('The sound of drums is heard from a distance. We are flying through mist and '
 'over the forest below. There is a hospital with a hole blown in, and we are '
 'glided by the drums building to a peak. A screech of brakes as a vechicle '
 'whines frammed; Jake Sully, a Scarred and Scuffy Combat Vet, sits in a '
 'beating up carbon fiber wheelchair. At 22 he eyes hardened by his wisdom '
 'andwariness of one whom has endured such a thing. Maglev Trails Whoosh '
 'overhead on raised tracks against a garish advertisementizing')


All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('There is a hospital with a hole blown in, and we are glided by the drums '
 'building to a peak. A screech of brakes as a vechicle whines frammed; Jake '
 'Sully, a Scarred and Scuffy Combat Vet, sits in a beating up carbon fiber '
 'wheelchair. At 22 he eyes hardened by his wisdom andwariness of one whom has '
 'endured such a thing. Maglev Trails Whoosh overhead on raised tracks against '
 'a garish advertisementizing')
{'Avatar': 'There is a hospital with a hole blown in, and we are glided by the drums building to a peak. A screech of brakes as a vechicle whines frammed; Jake Sully, a Scarred and Scuffy Combat Vet, sits in a beating up carbon fiber wheelchair. At 22 he eyes hardened by his wisdom andwariness of one whom has endured such a thing. Maglev Trails Whoosh overhead on raised tracks against a garish advertisementizing', 'two_stage_dialog_pegasus_short': {...}, 'lemma_pegasus_short': "Jake's voice is so strong it's hard to hear him , but I can see it in his eyes and his fa

## Display Results

In [ ]:
print(avatar_summary_dict)
print(avatar_rouge_dict)

{'Avatar': 'There is a hospital with a hole blown in, and we are glided by the drums building to a peak. A screech of brakes as a vechicle whines frammed; Jake Sully, a Scarred and Scuffy Combat Vet, sits in a beating up carbon fiber wheelchair. At 22 he eyes hardened by his wisdom andwariness of one whom has endured such a thing. Maglev Trails Whoosh overhead on raised tracks against a garish advertisementizing', 'two_stage_dialog_pegasus_short': {...}, 'lemma_pegasus_short': "Jake's voice is so strong it's hard to hear him , but I can see it in his eyes and his face and his eyes can't see me , so I can hear him in my head and my head can see him in his body and his body can see me in my body and my body can hear me in his mind and my mind can hear us in our body and our mind and we can hear each other's voices.", 'lemma_pegasus_long': "Combat vet Jake sully has been injured on duty in Iraq and Afghanistan .<n>He now lives in a wheelchair in a VA hospital in NYC and struggles to cope 

In [ ]:
avatar_summary_df = pd.DataFrame(avatar_summary_dict.items(), columns=['title', 'summary'])
avatar_summary_df.head()

,title,summary
0,Avatar,"There is a hospital with a hole blown in, and ..."
1,two_stage_dialog_pegasus_short,{'Avatar': 'There is a hospital with a hole bl...
2,lemma_pegasus_short,Jake's voice is so strong it's hard to hear hi...
3,lemma_pegasus_long,Combat vet Jake sully has been injured on duty...
4,lemma_pegasus_large,"we be very low over the forest now , glide fas..."


In [ ]:
avatar_rouge_df = pd.DataFrame(avatar_rouge_dict.items(), columns=['title', 'rouge'])
avatar_rouge_df.head()

,title,rouge
0,Avatar,"{'rouge1': 0.17621145374449337, 'rouge2': 0.01..."
1,two_stage_dialog_pegasus_short,"{'Avatar': {'rouge1': 0.17621145374449337, 'ro..."
2,lemma_pegasus_short,"{'rouge1': 0.14225941422594143, 'rouge2': 0.0,..."
3,lemma_pegasus_long,"{'rouge1': 0.2638297872340426, 'rouge2': 0.034..."
4,lemma_pegasus_large,"{'rouge1': 0.16289592760180996, 'rouge2': 0.02..."


In [ ]:
# save results to a csv
avatar_summary_df.to_csv('/content/avatar_summary.csv')
# files.download('/content/avatar_summary.csv') 

In [ ]:
# save results to a csv
avatar_rouge_df.to_csv('/content/avatar_rouge.csv')
# files.download('/content/avatar_rouge.csv') 